In [1]:
import sys
!{sys.executable} -m pip install --no-deps pygooglenews==0.1.2

!{sys.executable} -m pip install dateparser

!pip install numpy pandas yfinance 

!pip install git+https://github.com/aarigs/pandas-ta.git
    
!pip install xgboost lightgbm requests pytrends scikeras

!pip install GoogleNews pytrends transformers tqdm

  Cloning https://github.com/aarigs/pandas-ta.git to /tmp/pip-req-build-9beso6q2
  Running command git clone --filter=blob:none --quiet https://github.com/aarigs/pandas-ta.git /tmp/pip-req-build-9beso6q2
  Resolved https://github.com/aarigs/pandas-ta.git to commit 7a2a4210c71334929c482366d255d57eed5bdbfc
  Preparing metadata (setup.py) ... done
  Using cached scikeras-0.13.0-py3-none-any.whl (26 kB)
  Using cached keras-3.11.3-py3-none-any.whl (1.4 MB)


  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.11.3 which is incompatible.


In [3]:
import tensorflow as tf
print(tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
import ctypes
ctypes.CDLL('libcudnn.so.8')



2.15.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


<CDLL 'libcudnn.so.8', handle 787ff40 at 0x7f39b8690e20>

In [3]:
!pip install vaderSentiment

In [4]:
# 전체 파이프라인: 논문 재현 (TF-IDF OOF + VADER + Price + LSTM + CBR)
import os, glob, re, math, time
import numpy as np, pandas as pd
import yfinance as yf
from datetime import timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import TimeSeriesSplit
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# --------------------- 사용자 설정 ---------------------
ROOT_DIR = "./news_data"          
START_DATE = "2020-01-01"         
END_DATE = "2025-09-23"           
TRAIN_END = "2024-12-31"          
TICKER = "ETH-USD"                
SEQ_LEN = 10                      
MIN_TRAIN_DAYS = 60               
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE); tf.random.set_seed(RANDOM_STATE)

# --------------------- 진행상황 출력 유틸리티 ---------------------
def print_step(step_num, title, start_time=None):
    if start_time:
        elapsed = time.time() - start_time
        print(f"Step {step_num}: {title} - Completed in {elapsed:.2f} seconds")
    else:
        print(f"\nStep {step_num}: {title}")
        return time.time()

def print_progress_bar(current, total, prefix='Progress', length=50):
    percent = ("{0:.1f}").format(100 * (current / float(total)))
    filled_length = int(length * current // total)
    bar = '█' * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% ({current}/{total})', end='', flush=True)

# --------------------- 유틸: 파일에서 기사 읽기(모든 CSV) ---------------------
def read_articles_from_folder(folder):
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    print(f"Found {len(files)} CSV files in {folder}")
    
    rows=[]
    def parse_date_from_filename(fname):
        bn = os.path.basename(fname)
        m = re.search(r'(\d{4})[^\d](\d{2})[^\d](\d{2})', bn)
        if m:
            return pd.to_datetime(f"{m.group(1)}-{m.group(2)}-{m.group(3)}").normalize()
        return None
    
    processed_files = 0
    for f in files:
        try:
            df = pd.read_csv(f)
            processed_files += 1
            print_progress_bar(processed_files, len(files), 'Reading CSV files')
        except:
            continue
        
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date']).dt.normalize()
        else:
            dt = parse_date_from_filename(f)
            if dt is None:
                continue
            df['date'] = dt
        
        if 'news' not in df.columns:
            df['news'] = ""
        
        if 'label' in df.columns:
            df['label'] = pd.to_numeric(df['label'], errors='coerce')
        else:
            df['label'] = np.nan
        rows.append(df[['date','news','label']])
    
    print()  # New line after progress bar
    
    if not rows:
        return pd.DataFrame(columns=['date','news','label'])
    all_df = pd.concat(rows, ignore_index=True)
    all_df = all_df.sort_values('date').reset_index(drop=True)
    return all_df

# --------------------- VADER 점수 부여 ---------------------
analyzer = SentimentIntensityAnalyzer()
def add_vader_scores(article_df):
    total_articles = len(article_df)
    print(f"Computing VADER sentiment for {total_articles} articles...")
    
    processed = 0
    vader_scores = []
    word_counts = []
    
    for idx, text in enumerate(article_df['news'].astype(str)):
        vader_scores.append(analyzer.polarity_scores(text)['compound'])
        word_counts.append(len(text.split()))
        processed += 1
        
        if processed % 1000 == 0:
            print_progress_bar(processed, total_articles, 'VADER processing')
    
    print_progress_bar(total_articles, total_articles, 'VADER processing')
    print()
    
    article_df['vader_compound'] = vader_scores
    article_df['word_count'] = word_counts
    return article_df

# --------------------- TF-IDF OOF: expanding-window per date -> article-level positive prob OOF 생성
def article_oof_tfidf_probs(article_df, min_train_days=MIN_TRAIN_DAYS):
    adf = article_df.copy().reset_index(drop=True)
    unique_dates = adf['date'].drop_duplicates().sort_values().values
    n_dates = len(unique_dates)
    
    print(f"Computing TF-IDF OOF for {len(adf)} articles across {n_dates} unique dates")
    
    oof_pos = np.full(len(adf), np.nan)
    
    if n_dates <= min_train_days:
        print("Too few dates for proper OOF, fitting on all data")
        vect = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
        X = vect.fit_transform(adf['news'].astype(str).values)
        y = adf['label'].fillna(0).replace(-1,2).astype(int).values
        clf = LogisticRegression(solver='saga', max_iter=2000)
        clf.fit(X, y)
        if 1 in clf.classes_:
            pos_idx = np.where(clf.classes_==1)[0][0]
            oof_pos[:] = clf.predict_proba(X)[:,pos_idx]
        else:
            oof_pos[:] = 0.0
        adf['tfidf_pos_prob_oof'] = oof_pos
        return adf

    processed_dates = 0
    for i in range(min_train_days, n_dates):
        train_dates = unique_dates[:i]
        val_dates = np.array([unique_dates[i]])
        train_idx = adf[adf['date'].isin(train_dates)].index.values
        val_idx = adf[adf['date'].isin(val_dates)].index.values
        
        if train_idx.size==0 or val_idx.size==0:
            continue
            
        vect = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
        Xtr = vect.fit_transform(adf.loc[train_idx,'news'].astype(str).values)
        ytr = adf.loc[train_idx,'label'].fillna(0).replace(-1,2).astype(int).values
        
        clf = LogisticRegression(solver='saga', max_iter=2000, class_weight='balanced')
        clf.fit(Xtr, ytr)
        Xval = vect.transform(adf.loc[val_idx,'news'].astype(str).values)
        probs = clf.predict_proba(Xval)
        
        if 1 in clf.classes_:
            pos_idx = np.where(clf.classes_==1)[0][0]
            oof_pos[val_idx] = probs[:,pos_idx]
        else:
            oof_pos[val_idx] = 0.0
            
        processed_dates += 1
        if processed_dates % 10 == 0:
            print_progress_bar(processed_dates, n_dates - min_train_days, 'TF-IDF OOF')
    
    print_progress_bar(n_dates - min_train_days, n_dates - min_train_days, 'TF-IDF OOF')
    print()
    
    # Fill remaining nan
    nan_idx = np.where(np.isnan(oof_pos))[0]
    if nan_idx.size>0:
        print(f"Filling {nan_idx.size} NaN values with initial model")
        first_dates = unique_dates[:min_train_days]
        train_idx0 = adf[adf['date'].isin(first_dates)].index.values
        if train_idx0.size == 0:
            oof_pos[nan_idx] = 0.0
        else:
            vect = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
            Xtr = vect.fit_transform(adf.loc[train_idx0,'news'].astype(str).values)
            ytr = adf.loc[train_idx0,'label'].fillna(0).replace(-1,2).astype(int).values
            clf = LogisticRegression(solver='saga', max_iter=2000, class_weight='balanced')
            clf.fit(Xtr, ytr)
            Xrem = vect.transform(adf.loc[nan_idx,'news'].astype(str).values)
            probs = clf.predict_proba(Xrem)
            if 1 in clf.classes_:
                pos_idx = np.where(clf.classes_==1)[0][0]
                oof_pos[nan_idx] = probs[:,pos_idx]
            else:
                oof_pos[nan_idx] = 0.0

    adf['tfidf_pos_prob_oof'] = oof_pos
    return adf

# --------------------- 일별 집계 (논문과 동일한 형태) ---------------------
def daily_aggregate_from_articles(article_df):
    print("Aggregating articles by date...")
    g = article_df.groupby('date')
    agg = g.agg(
        news_count = ('news','count'),
        tfidf_pos_mean = ('tfidf_pos_prob_oof','mean'),
        tfidf_pos_sum = ('tfidf_pos_prob_oof','sum'),
        tfidf_pos_std = ('tfidf_pos_prob_oof','std'),
        vader_mean = ('vader_compound','mean'),
        vader_sum = ('vader_compound','sum'),
        vader_std = ('vader_compound','std'),
        label_mean = ('label','mean'),
        avg_body_len = ('word_count','mean')
    ).reset_index()
    
    pos = g['label'].apply(lambda s: (s==1).sum())
    neg = g['label'].apply(lambda s: (s==-1).sum())
    neu = g['label'].apply(lambda s: (s==0).sum())
    agg['pos_ratio'] = pos.values / agg['news_count']
    agg['neg_ratio'] = neg.values / agg['news_count']
    agg['neu_ratio'] = neu.values / agg['news_count']
    agg[['tfidf_pos_std','vader_std']] = agg[['tfidf_pos_std','vader_std']].fillna(0)
    
    agg = agg.sort_values('date').reset_index(drop=True)
    print(f"Created daily aggregation for {len(agg)} days")
    return agg

# --------------------- 시간 파생변수: rolling, EWMA, zscore, lag ---------------------
def add_time_features(daily_df):
    print("Computing time-based features...")
    df = daily_df.sort_values('date').reset_index(drop=True)
    
    for w in [3,7,14]:
        df[f'roll_tfidf_mean_{w}'] = df['tfidf_pos_mean'].rolling(window=w, min_periods=1).mean()
        df[f'roll_tfidf_std_{w}'] = df['tfidf_pos_mean'].rolling(window=w, min_periods=1).std().fillna(0)
        df[f'roll_news_count_{w}'] = df['news_count'].rolling(window=w, min_periods=1).sum()
    
    df['tfidf_ewma'] = df['tfidf_pos_mean'].ewm(alpha=0.3, adjust=False).mean()
    
    mu = df['tfidf_pos_mean'].rolling(window=7, min_periods=1).mean()
    sd = df['tfidf_pos_mean'].rolling(window=7, min_periods=1).std().replace(0, np.nan).fillna(1e-8)
    df['tfidf_z'] = (df['tfidf_pos_mean'] - mu) / sd
    
    lag_cols = df.columns.difference(['date']).tolist()
    news_cols = ['news_count','tfidf_pos_sum','tfidf_pos_mean','tfidf_pos_std','vader_mean','vader_sum','vader_std','pos_ratio','neg_ratio','neu_ratio','avg_body_len',
                 'tfidf_ewma','tfidf_z'] + [f'roll_tfidf_mean_{w}' for w in [3,7,14]] + [f'roll_tfidf_std_{w}' for w in [3,7,14]] + [f'roll_news_count_{w}' for w in [3,7,14]]
    news_cols = [c for c in news_cols if c in df.columns]
    
    for c in news_cols:
        df[f'{c}_lag1'] = df[c].shift(1).fillna(0)
    
    print(f"Added {sum(1 for c in df.columns if c.endswith('_lag1'))} lag features")
    return df

# --------------------- 가격 데이터 수집 및 타깃 생성 ---------------------
def fetch_price_data(start_date, end_date, ticker=TICKER):
    print(f"Fetching price data for {ticker} from {start_date} to {end_date}")
    df = yf.download(ticker, start=start_date, end=(pd.to_datetime(end_date)+pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False, auto_adjust=False)
    
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.reset_index().rename(columns={'Date':'date'})
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    
    close_col = next((c for c in df.columns if 'close' in str(c).lower()), None)
    if close_col is None:
        raise RuntimeError("No close column in price data")
    df = df[['date', close_col]].rename(columns={close_col:'close'})
    df = df[df['date'] >= pd.to_datetime(START_DATE)].reset_index(drop=True)
    
    df['ret'] = df['close'].pct_change()
    df['target'] = (df['ret'] > 0).astype(int)
    
    for l in [1,2,3]:
        df[f'ret_lag_{l}'] = df['ret'].shift(l)
    
    print(f"Price data: {len(df)} days, target distribution: {df['target'].value_counts().to_dict()}")
    return df

# --------------------- 시퀀스 생성 유틸 ---------------------
def make_sequences(arr, lookback):
    Xs=[]
    for i in range(lookback, len(arr)):
        Xs.append(arr[i-lookback:i])
    return np.array(Xs)

# --------------------- LSTM 빌드 ---------------------
def build_lstm_model(input_dim, units=32):
    m = Sequential([LSTM(units, input_shape=(SEQ_LEN, input_dim)), Dropout(0.2), Dense(1, activation='sigmoid')])
    m.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return m

# --------------------- train-only seq scaler ---------------------
def scale_seq_train_only(X_seq, train_seq_mask):
    n,s,f = X_seq.shape
    flat_train = X_seq[train_seq_mask].reshape(-1, f)
    sc = StandardScaler().fit(flat_train)
    scaled = sc.transform(X_seq.reshape(-1, f)).reshape(n, s, f)
    return scaled, sc

# --------------------- Base-model OOF (일별) for CBR용 --------------------------------
def oof_prob_by_timeseries(X, y, dates, model_factory, min_train_days=MIN_TRAIN_DAYS):
    print(f"Computing OOF probabilities using expanding window (min_train_days={min_train_days})")
    n = len(dates)
    oof = np.full(n, np.nan)
    unique_dates = np.array(pd.Series(dates).dt.normalize().drop_duplicates())
    
    if len(unique_dates) <= min_train_days:
        m = model_factory()
        m.fit(X, y)
        if hasattr(m, "predict_proba"):
            oof[:] = m.predict_proba(X)[:,1]
        else:
            oof[:] = m.predict(X)
        return oof

    processed = 0
    total_folds = len(unique_dates) - min_train_days
    
    for i in range(min_train_days, len(unique_dates)):
        train_dates = unique_dates[:i]
        val_dates = np.array([unique_dates[i]])
        train_idx = np.where(pd.Series(dates).isin(train_dates))[0]
        val_idx = np.where(pd.Series(dates).isin(val_dates))[0]
        
        if train_idx.size==0 or val_idx.size==0:
            continue
            
        Xtr, ytr = X[train_idx], y[train_idx]
        Xval = X[val_idx]
        m = model_factory()
        m.fit(Xtr, ytr)
        
        if hasattr(m, "predict_proba"):
            oof[val_idx] = m.predict_proba(Xval)[:,1]
        else:
            oof[val_idx] = m.predict(Xval)
        
        processed += 1
        if processed % 5 == 0:
            print_progress_bar(processed, total_folds, 'OOF computation')
    
    print_progress_bar(total_folds, total_folds, 'OOF computation')
    print()
    
    nan_idx = np.where(np.isnan(oof))[0]
    if nan_idx.size>0:
        first_dates = unique_dates[:min_train_days]
        train_idx0 = np.where(pd.Series(dates).isin(first_dates))[0]
        if train_idx0.size>0:
            Xtr0, ytr0 = X[train_idx0], y[train_idx0]
            m0 = model_factory()
            m0.fit(Xtr0, ytr0)
            if hasattr(m0, "predict_proba"):
                oof[nan_idx] = m0.predict_proba(X[nan_idx])[:,1]
            else:
                oof[nan_idx] = m0.predict(X[nan_idx])
        else:
            oof[nan_idx] = 0.0
    return oof

# --------------------- LSTM OOF (시퀀스 단위) - 간단 expanding-loop 방식 (비용 큼) ----------
def lstm_oof_sequences(X_seq, y_seq, seq_dates, build_model_fn, epochs=5, batch_size=64, min_train_seq=30):
    print(f"Computing LSTM OOF for {X_seq.shape[0]} sequences (this may take a while...)")
    n_seq = X_seq.shape[0]
    oof = np.full(n_seq, np.nan)
    
    if n_seq <= min_train_seq:
        m = build_model_fn()
        m.fit(X_seq, y_seq, epochs=epochs, batch_size=batch_size, verbose=0)
        oof[:] = m.predict(X_seq)[:,0]
        return oof

    processed = 0
    total_seqs = n_seq - min_train_seq
    
    for i in range(min_train_seq, n_seq):
        train_idx = np.arange(0, i)
        val_idx = np.array([i])
        Xtr, ytr = X_seq[train_idx], y_seq[train_idx]
        Xval = X_seq[val_idx]
        m = build_model_fn()
        m.fit(Xtr, ytr, epochs=epochs, batch_size=batch_size, verbose=0)
        oof[val_idx] = m.predict(Xval)[:,0]
        
        processed += 1
        if processed % 10 == 0:
            print_progress_bar(processed, total_seqs, 'LSTM OOF')
    
    print_progress_bar(total_seqs, total_seqs, 'LSTM OOF')
    print()
    
    nan_idx = np.where(np.isnan(oof))[0]
    if nan_idx.size>0:
        m0 = build_model_fn()
        m0.fit(X_seq[:min_train_seq], y_seq[:min_train_seq], epochs=epochs, batch_size=batch_size, verbose=0)
        for idx in nan_idx:
            oof[idx] = m0.predict(X_seq[idx:idx+1])[:,0]
    return oof

# --------------------- 메트릭 함수 ---------------------
def compute_metrics(y_true, y_pred, y_prob=None):
    mask = ~pd.isna(y_pred)
    if mask.sum()==0:
        return {'Accuracy':np.nan,'Precision':np.nan,'Recall':np.nan,'F1':np.nan,'AUC':np.nan,'MDA':np.nan,'MDE':np.nan}
    y_t = y_true[mask].astype(int); y_p = y_pred[mask].astype(int)
    out={}
    out['Accuracy']=accuracy_score(y_t,y_p)
    out['Precision']=precision_score(y_t,y_p,zero_division=0)
    out['Recall']=recall_score(y_t,y_p,zero_division=0)
    out['F1']=f1_score(y_t,y_p,zero_division=0)
    out['AUC']=roc_auc_score(y_t,y_prob[mask]) if (y_prob is not None and len(np.unique(y_t))>1) else np.nan
    out['MDA']=out['Accuracy']; out['MDE']=np.mean(y_p != y_t)
    return out

# --------------------- 파이프라인 실행 ---------------------
if __name__ == "__main__":
    pipeline_start = time.time()
    print("=" * 60)
    print("PAPER REPRODUCTION PIPELINE STARTING")
    print("=" * 60)
    
    # 1) 기사 읽기
    start_time = print_step(1, "Reading articles from CSV files")
    articles = read_articles_from_folder(ROOT_DIR)
    print_step(1, f"Articles loaded: {len(articles)} articles", start_time)
    print(f"Date range: {articles['date'].min()} to {articles['date'].max()}")
    print(f"Label distribution: {articles['label'].value_counts().to_dict()}")

    # 2) VADER
    start_time = print_step(2, "Computing VADER sentiment scores")
    articles = add_vader_scores(articles)
    print_step(2, "VADER sentiment computation completed", start_time)

    # 3) TF-IDF OOF (article-level)
    start_time = print_step(3, "Computing TF-IDF OOF probabilities")
    articles = article_oof_tfidf_probs(articles, min_train_days=MIN_TRAIN_DAYS)
    print_step(3, "TF-IDF OOF computation completed", start_time)

    # 4) daily aggregate
    start_time = print_step(4, "Daily aggregation and feature engineering")
    daily = daily_aggregate_from_articles(articles)
    daily = add_time_features(daily)
    print_step(4, f"Daily features created: {len(daily)} days, {daily.shape[1]} features", start_time)

    # 5) fetch price and merge
    start_time = print_step(5, "Fetching price data and merging")
    price = fetch_price_data(START_DATE, END_DATE, ticker=TICKER)
    merged = pd.merge(price, daily, on='date', how='left').sort_values('date').reset_index(drop=True)
    
    news_feature_cols = [c for c in merged.columns if any(s in c for s in ['tfidf_pos','vader','pos_ratio','neg_ratio','neu_ratio','avg_body_len','roll_tfidf','tfidf_ewma','tfidf_z','news_count'])]
    merged[news_feature_cols] = merged[news_feature_cols].fillna(0)
    print_step(5, f"Data merged: {len(merged)} days with {len(news_feature_cols)} news features", start_time)

    # 6) features set
    start_time = print_step(6, "Feature selection and dataset preparation")
    price_feats = [f'ret_lag_{i}' for i in [1,2,3] if f'ret_lag_{i}' in merged.columns]
    news_lag_feats = [c for c in merged.columns if c.endswith('_lag1')]
    news_lag_feats = [c for c in news_lag_feats if any(x in c for x in ['tfidf_pos_mean','tfidf_pos_sum','tfidf_pos_std','vader_mean','pos_ratio','news_count','tfidf_ewma','tfidf_z','avg_body_len','roll_news_count_3','roll_news_count_7','roll_news_count_14'])]
    all_feats = price_feats + news_lag_feats

    # 7) form dataset and drop NaNs
    data = merged[['date'] + all_feats + ['target']].dropna().reset_index(drop=True)
    train_mask = data['date'] <= pd.to_datetime(TRAIN_END)
    test_mask = data['date'] > pd.to_datetime(TRAIN_END)
    print_step(6, f"Dataset prepared: {data.shape[0]} rows, {len(all_feats)} features", start_time)
    print(f"Train/Test split: {train_mask.sum()}/{test_mask.sum()} days")

    # 8) scale train-only for base models
    start_time = print_step(7, "Feature scaling")
    X_train = data.loc[train_mask, all_feats].values
    X_test = data.loc[test_mask, all_feats].values
    y_train = data.loc[train_mask, 'target'].values
    y_test = data.loc[test_mask, 'target'].values
    
    scaler = StandardScaler().fit(X_train)
    X_train_s = scaler.transform(X_train)
    X_test_s = scaler.transform(X_test)

    Xp_train = data.loc[train_mask, price_feats].values
    Xp_test = data.loc[test_mask, price_feats].values
    scaler_p = StandardScaler().fit(Xp_train)
    Xp_train_s = scaler_p.transform(Xp_train)
    Xp_test_s = scaler_p.transform(Xp_test)

    Xn_train = data.loc[train_mask, all_feats].values
    Xn_test = data.loc[test_mask, all_feats].values
    scaler_n = StandardScaler().fit(Xn_train)
    Xn_train_s = scaler_n.transform(Xn_train)
    Xn_test_s = scaler_n.transform(Xn_test)
    print_step(7, "Feature scaling completed", start_time)

    # 9) Base models train
    start_time = print_step(8, "Training base models")
    def make_logit(): return LogisticRegression(max_iter=2000, class_weight='balanced')
    def make_ann(): return MLPClassifier(hidden_layer_sizes=(10,), max_iter=2000)
    def make_svm(): return SVC(kernel='rbf', probability=True)

    print("Training price-only models...")
    mdl_price_logit = make_logit().fit(Xp_train_s, y_train)
    mdl_price_ann = make_ann().fit(Xp_train_s, y_train)
    mdl_price_svm = make_svm().fit(Xp_train_s, y_train)

    print("Training news-based models...")
    mdl_news_logit = make_logit().fit(Xn_train_s, y_train)
    mdl_news_ann = make_ann().fit(Xn_train_s, y_train)
    mdl_news_svm = make_svm().fit(Xn_train_s, y_train)
    print_step(8, "Base models training completed", start_time)

    # 10) OOF probs for news-based daily models
    start_time = print_step(9, "Computing OOF probabilities for CBR")
    X_all_days = data[all_feats].values
    y_all_days = data['target'].values
    dates_all = data['date']
    
    print("Computing Logistic Regression OOF...")
    oof_logit = oof_prob_by_timeseries(X_all_days, y_all_days, dates_all, make_logit, min_train_days=MIN_TRAIN_DAYS)
    print("Computing ANN OOF...")
    oof_ann = oof_prob_by_timeseries(X_all_days, y_all_days, dates_all, make_ann, min_train_days=MIN_TRAIN_DAYS)
    print("Computing SVM OOF...")
    oof_svm = oof_prob_by_timeseries(X_all_days, y_all_days, dates_all, make_svm, min_train_days=MIN_TRAIN_DAYS)
    print_step(9, "OOF probabilities computation completed", start_time)

    # 11) LSTM: prepare sequences
    start_time = print_step(10, "Preparing LSTM sequences and training")
    full_feats = price_feats + news_lag_feats
    full_array = data[full_feats].values
    seq_X = make_sequences(full_array, SEQ_LEN)
    seq_y = data['target'].values[SEQ_LEN:]
    seq_dates = data['date'].values[SEQ_LEN:]
    
    print(f"Created {seq_X.shape[0]} sequences of length {SEQ_LEN} with {seq_X.shape[2]} features")

    seq_train_mask = seq_dates <= np.datetime64(TRAIN_END)
    seq_X_s, seq_scaler = scale_seq_train_only(seq_X, seq_train_mask)
    
    input_dim = seq_X_s.shape[2]
    print(f"Training LSTM model (input_dim={input_dim}, train_sequences={seq_train_mask.sum()})...")
    model_lstm = build_lstm_model(input_dim, units=32)
    model_lstm.fit(seq_X_s[seq_train_mask], seq_y[seq_train_mask], epochs=10, batch_size=64, verbose=0)
    
    seq_test_mask = seq_dates > np.datetime64(TRAIN_END)
    prob_lstm_test = model_lstm.predict(seq_X_s[seq_test_mask])[:,0]
    pred_lstm_test = (prob_lstm_test > 0.5).astype(int)
    print_step(10, "LSTM model training completed", start_time)

    # 12) LSTM OOF for seqs (for CBR)
    start_time = print_step(11, "Computing LSTM OOF (expensive operation)")
    lstm_oof_probs = lstm_oof_sequences(seq_X_s, seq_y, seq_dates, lambda: build_lstm_model(input_dim, units=32), epochs=5, batch_size=64, min_train_seq=30)
    
    n_days = len(data)
    lstm_day_oof = np.concatenate([np.full(SEQ_LEN, np.nan), lstm_oof_probs])
    print_step(11, "LSTM OOF computation completed", start_time)

    # 13) Build case-probs matrix for CBR
    start_time = print_step(12, "Building CBR case probability matrix")
    full_news_X = data[all_feats].values
    full_scaled = scaler_n.transform(full_news_X)
    full_prob_logit = mdl_news_logit.predict_proba(full_scaled)[:,1]
    full_prob_ann = mdl_news_ann.predict_proba(full_scaled)[:,1]
    full_prob_svm = mdl_news_svm.predict_proba(full_scaled)[:,1]
    
    seq_preds_full = np.concatenate([np.full(SEQ_LEN, np.nan), model_lstm.predict(seq_X_s)[:,0]])
    full_probs = np.vstack([full_prob_logit, full_prob_ann, full_prob_svm, seq_preds_full]).T

    seq_start = SEQ_LEN
    train_case_idx = np.where((data['date'].values[seq_start:] <= np.datetime64(TRAIN_END)))[0] + seq_start
    test_case_idx  = np.where((data['date'].values[seq_start:] >  np.datetime64(TRAIN_END)))[0] + seq_start

    oof_matrix = np.vstack([oof_logit, oof_ann, oof_svm, lstm_day_oof]).T
    case_probs_train = oof_matrix[train_case_idx]
    case_probs_test  = full_probs[test_case_idx]
    y_train_case = data['target'].values[train_case_idx]
    y_test_case  = data['target'].values[test_case_idx]
    print_step(12, f"CBR matrix built: {case_probs_train.shape[0]} train cases, {case_probs_test.shape[0]} test cases", start_time)

    # 14) CBR k selection
    start_time = print_step(13, "CBR hyperparameter tuning")
    n_train_case = case_probs_train.shape[0]
    if n_train_case < 10:
        best_k = 1
        print(f"Too few training cases ({n_train_case}), using k=1")
    else:
        split = int(n_train_case * 0.8)
        tr_idx = np.arange(0, split)
        val_idx = np.arange(split, n_train_case)
        best_k, best_acc = 1, -1
        
        print("Testing k values from 1 to 10...")
        for k in range(1, 11):
            neigh = NearestNeighbors(n_neighbors=k).fit(case_probs_train[tr_idx])
            _, idxs = neigh.kneighbors(case_probs_train[val_idx])
            preds = np.array([int(np.round(y_train_case[tr_idx][idxs[i]].mean())) for i in range(len(idxs))])
            acc = accuracy_score(y_train_case[val_idx], preds)
            if acc > best_acc:
                best_acc = acc; best_k = k
        print(f"Best k={best_k} with validation accuracy={best_acc:.4f}")

    neigh = NearestNeighbors(n_neighbors=best_k).fit(case_probs_train)
    _, idxs_test = neigh.kneighbors(case_probs_test)
    preds_test = np.array([int(np.round(y_train_case[idxs_test[i]].mean())) for i in range(len(idxs_test))])
    print_step(13, "CBR prediction completed", start_time)

    # 15) Evaluate models
    start_time = print_step(14, "Model evaluation")
    rows=[]
    
    print("Evaluating price-only models...")
    Xp_full = data[price_feats].values
    Xp_test_full = scaler_p.transform(Xp_full[test_mask.values])
    for name, m in [('Logit', mdl_price_logit), ('ANN', mdl_price_ann), ('SVM', mdl_price_svm)]:
        prob = m.predict_proba(Xp_test_full)[:,1] if hasattr(m, "predict_proba") else m.predict(Xp_test_full)
        pred = (prob>0.5).astype(int)
        rows.append({'Model':f'P_{name}', **compute_metrics(y_test, pred, prob)})

    print("Evaluating news-based models...")
    Xn_full_scaled = scaler_n.transform(data[all_feats].values)
    Xn_test_full = Xn_full_scaled[test_mask.values]
    for name, m in [('Logit', mdl_news_logit), ('ANN', mdl_news_ann), ('SVM', mdl_news_svm)]:
        prob = m.predict_proba(Xn_test_full)[:,1]
        pred = (prob>0.5).astype(int)
        rows.append({'Model':f'N_{name}', **compute_metrics(y_test, pred, prob)})

    print("Evaluating LSTM...")
    rows.append({'Model':'N_LSTM', **compute_metrics(y_test[seq_test_mask], pred_lstm_test, prob_lstm_test)})

    print("Evaluating CBR...")
    cbr_pred_full = np.full(test_mask.sum(), np.nan)
    test_global_idx = np.where(test_mask.values)[0]
    for i, gidx in enumerate(test_case_idx):
        pos = np.where(test_global_idx==gidx)[0]
        if pos.size>0:
            cbr_pred_full[pos[0]] = preds_test[i]
    rows.append({'Model':'N_CBR', **compute_metrics(y_test, cbr_pred_full, None)})

    res_df = pd.DataFrame(rows).set_index('Model').round(4)
    print_step(14, "Model evaluation completed", start_time)
    
    print("\n" + "=" * 60)
    print("FINAL RESULTS")
    print("=" * 60)
    print(res_df.sort_values('Accuracy', ascending=False))

    # 16) confusion matrices 출력
    print("\n" + "=" * 60)
    print("CONFUSION MATRICES FOR TOP MODELS")
    print("=" * 60)
    
    top_models = res_df.sort_values('Accuracy', ascending=False).head(3)
    for idx in top_models.index:
        print(f"\nModel: {idx} (Accuracy: {top_models.loc[idx, 'Accuracy']:.4f})")
        if idx.startswith('P_'):
            name = idx.split('_',1)[1]
            if name=='Logit':
                prob = mdl_price_logit.predict_proba(Xp_test_s)[:,1]; pred = (prob>0.5).astype(int)
            elif name=='ANN':
                prob = mdl_price_ann.predict_proba(Xp_test_s)[:,1]; pred = (prob>0.5).astype(int)
            else:
                prob = mdl_price_svm.predict_proba(Xp_test_s)[:,1]; pred = (prob>0.5).astype(int)
            print('Confusion Matrix:')
            print(confusion_matrix(y_test, pred))
        elif idx.startswith('N_') and idx!='N_CBR':
            name = idx.split('_',1)[1]
            if name=='LSTM':
                print('Confusion Matrix (seq-aligned):')
                print(confusion_matrix(y_test[seq_test_mask], pred_lstm_test))
            else:
                if name=='Logit':
                    prob = mdl_news_logit.predict_proba(Xn_test_full)[:,1]; pred = (prob>0.5).astype(int)
                elif name=='ANN':
                    prob = mdl_news_ann.predict_proba(Xn_test_full)[:,1]; pred = (prob>0.5).astype(int)
                else:
                    prob = mdl_news_svm.predict_proba(Xn_test_full)[:,1]; pred = (prob>0.5).astype(int)
                print('Confusion Matrix:')
                print(confusion_matrix(y_test, pred))
        else:
            print('Confusion Matrix:')
            valid_mask = ~pd.isna(cbr_pred_full)
            if valid_mask.sum() > 0:
                print(confusion_matrix(y_test[valid_mask], cbr_pred_full[valid_mask]))
            else:
                print("No valid CBR predictions")

    total_time = time.time() - pipeline_start
    print(f"\n" + "=" * 60)
    print(f"PIPELINE COMPLETED IN {total_time:.2f} SECONDS")
    print("=" * 60)

PAPER REPRODUCTION PIPELINE STARTING

Step 1: Reading articles from CSV files
Found 2093 CSV files in ./news_data
Reading CSV files |█████████████████████████████████████████████████-| 100.0% (2092/2093)
Step 1: Articles loaded: 25774 articles - Completed in 12.03 seconds
Date range: 2020-01-01 00:00:00 to 2025-09-22 00:00:00
Label distribution: {1: 12304, 0: 8106, -1: 5364}

Step 2: Computing VADER sentiment scores
Computing VADER sentiment for 25774 articles...
VADER processing |██████████████████████████████████████████████████| 100.0% (25774/25774)
Step 2: VADER sentiment computation completed - Completed in 8.64 seconds

Step 3: Computing TF-IDF OOF probabilities
Computing TF-IDF OOF for 25774 articles across 2050 unique dates
TF-IDF OOF |██████████████████████████████████████████████████| 100.0% (1990/1990)
Filling 174 NaN values with initial model
Step 3: TF-IDF OOF computation completed - Completed in 9647.08 seconds

Step 4: Daily aggregation and feature engineering
Aggregatin

/raid/invigoworks/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


OOF computation |████████████████████████--------------------------| 49.3% (1000/2029)

/raid/invigoworks/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


OOF computation |█████████████████████████████████-----------------| 66.0% (1340/2029)

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step30/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step40/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/ste

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step50/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step60/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step70/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step0/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step90/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step00/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step10/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/ste

IndexError: boolean index did not match indexed array along dimension 0; dimension is 266 but corresponding boolean dimension is 2079

In [1]:
# 현재 상태에서 변수들을 확인해보고
print("seq_y 존재:", 'seq_y' in locals())
print("model_lstm 존재:", 'model_lstm' in locals())

# 만약 없다면
globals().update(locals())  # 모든 로컬 변수를 글로벌로 이동

seq_y 존재: False
model_lstm 존재: False


In [ ]:
# 12) LSTM OOF for seqs (for CBR)
start_time = print_step(11, "Computing LSTM OOF (expensive operation)")
lstm_oof_probs = lstm_oof_sequences(seq_X_s, seq_y, seq_dates, lambda: build_lstm_model(input_dim, units=32), epochs=5, batch_size=64, min_train_seq=30)

n_days = len(data)
lstm_day_oof = np.concatenate([np.full(SEQ_LEN, np.nan), lstm_oof_probs])
print_step(11, "LSTM OOF computation completed", start_time)

# 13) Build case-probs matrix for CBR
start_time = print_step(12, "Building CBR case probability matrix")
full_news_X = data[all_feats].values
full_scaled = scaler_n.transform(full_news_X)
full_prob_logit = mdl_news_logit.predict_proba(full_scaled)[:,1]
full_prob_ann = mdl_news_ann.predict_proba(full_scaled)[:,1]
full_prob_svm = mdl_news_svm.predict_proba(full_scaled)[:,1]

seq_preds_full = np.concatenate([np.full(SEQ_LEN, np.nan), model_lstm.predict(seq_X_s)[:,0]])
full_probs = np.vstack([full_prob_logit, full_prob_ann, full_prob_svm, seq_preds_full]).T

seq_start = SEQ_LEN
train_case_idx = np.where((data['date'].values[seq_start:] <= np.datetime64(TRAIN_END)))[0] + seq_start
test_case_idx  = np.where((data['date'].values[seq_start:] >  np.datetime64(TRAIN_END)))[0] + seq_start

oof_matrix = np.vstack([oof_logit, oof_ann, oof_svm, lstm_day_oof]).T
case_probs_train = oof_matrix[train_case_idx]
case_probs_test  = full_probs[test_case_idx]
y_train_case = data['target'].values[train_case_idx]
y_test_case  = data['target'].values[test_case_idx]
print_step(12, f"CBR matrix built: {case_probs_train.shape[0]} train cases, {case_probs_test.shape[0]} test cases", start_time)

# 14) CBR k selection
start_time = print_step(13, "CBR hyperparameter tuning")
n_train_case = case_probs_train.shape[0]
if n_train_case < 10:
    best_k = 1
    print(f"Too few training cases ({n_train_case}), using k=1")
else:
    split = int(n_train_case * 0.8)
    tr_idx = np.arange(0, split)
    val_idx = np.arange(split, n_train_case)
    best_k, best_acc = 1, -1

    print("Testing k values from 1 to 10...")
    for k in range(1, 11):
        neigh = NearestNeighbors(n_neighbors=k).fit(case_probs_train[tr_idx])
        _, idxs = neigh.kneighbors(case_probs_train[val_idx])
        preds = np.array([int(np.round(y_train_case[tr_idx][idxs[i]].mean())) for i in range(len(idxs))])
        acc = accuracy_score(y_train_case[val_idx], preds)
        if acc > best_acc:
            best_acc = acc; best_k = k
    print(f"Best k={best_k} with validation accuracy={best_acc:.4f}")

neigh = NearestNeighbors(n_neighbors=best_k).fit(case_probs_train)
_, idxs_test = neigh.kneighbors(case_probs_test)
preds_test = np.array([int(np.round(y_train_case[idxs_test[i]].mean())) for i in range(len(idxs_test))])
print_step(13, "CBR prediction completed", start_time)

# 15) Evaluate models
start_time = print_step(14, "Model evaluation")
rows=[]

print("Evaluating price-only models...")
Xp_full = data[price_feats].values
Xp_test_full = scaler_p.transform(Xp_full[test_mask.values])
for name, m in [('Logit', mdl_price_logit), ('ANN', mdl_price_ann), ('SVM', mdl_price_svm)]:
    prob = m.predict_proba(Xp_test_full)[:,1] if hasattr(m, "predict_proba") else m.predict(Xp_test_full)
    pred = (prob>0.5).astype(int)
    rows.append({'Model':f'P_{name}', **compute_metrics(y_test, pred, prob)})

print("Evaluating news-based models...")
Xn_full_scaled = scaler_n.transform(data[all_feats].values)
Xn_test_full = Xn_full_scaled[test_mask.values]
for name, m in [('Logit', mdl_news_logit), ('ANN', mdl_news_ann), ('SVM', mdl_news_svm)]:
    prob = m.predict_proba(Xn_test_full)[:,1]
    pred = (prob>0.5).astype(int)
    rows.append({'Model':f'N_{name}', **compute_metrics(y_test, pred, prob)})

print("Evaluating LSTM...")
rows.append({'Model':'N_LSTM', **compute_metrics(seq_y[seq_test_mask], pred_lstm_test, prob_lstm_test)})

print("Evaluating CBR...")
cbr_pred_full = np.full(test_mask.sum(), np.nan)
test_global_idx = np.where(test_mask.values)[0]
for i, gidx in enumerate(test_case_idx):
    pos = np.where(test_global_idx==gidx)[0]
    if pos.size>0:
        cbr_pred_full[pos[0]] = preds_test[i]
rows.append({'Model':'N_CBR', **compute_metrics(y_test, cbr_pred_full, None)})

res_df = pd.DataFrame(rows).set_index('Model').round(4)
print_step(14, "Model evaluation completed", start_time)

print("\n" + "=" * 60)
print("FINAL RESULTS")
print("=" * 60)
print(res_df.sort_values('Accuracy', ascending=False))

# 16) confusion matrices 출력
print("\n" + "=" * 60)
print("CONFUSION MATRICES FOR TOP MODELS")
print("=" * 60)

top_models = res_df.sort_values('Accuracy', ascending=False).head(3)
for idx in top_models.index:
    print(f"\nModel: {idx} (Accuracy: {top_models.loc[idx, 'Accuracy']:.4f})")
    if idx.startswith('P_'):
        name = idx.split('_',1)[1]
        if name=='Logit':
            prob = mdl_price_logit.predict_proba(Xp_test_s)[:,1]; pred = (prob>0.5).astype(int)
        elif name=='ANN':
            prob = mdl_price_ann.predict_proba(Xp_test_s)[:,1]; pred = (prob>0.5).astype(int)
        else:
            prob = mdl_price_svm.predict_proba(Xp_test_s)[:,1]; pred = (prob>0.5).astype(int)
        print('Confusion Matrix:')
        print(confusion_matrix(y_test, pred))
    elif idx.startswith('N_') and idx!='N_CBR':
        name = idx.split('_',1)[1]
        if name=='LSTM':
            print('Confusion Matrix (seq-aligned):')
            print(confusion_matrix(seq_y[seq_test_mask], pred_lstm_test))
        else:
            if name=='Logit':
                prob = mdl_news_logit.predict_proba(Xn_test_full)[:,1]; pred = (prob>0.5).astype(int)
            elif name=='ANN':
                prob = mdl_news_ann.predict_proba(Xn_test_full)[:,1]; pred = (prob>0.5).astype(int)
            else:
                prob = mdl_news_svm.predict_proba(Xn_test_full)[:,1]; pred = (prob>0.5).astype(int)
            print('Confusion Matrix:')
            print(confusion_matrix(y_test, pred))
    else:
        print('Confusion Matrix:')
        valid_mask = ~pd.isna(cbr_pred_full)
        if valid_mask.sum() > 0:
            print(confusion_matrix(y_test[valid_mask], cbr_pred_full[valid_mask]))
        else:
            print("No valid CBR predictions")

    total_time = time.time() - pipeline_start
    print(f"\n" + "=" * 60)
    print(f"PIPELINE COMPLETED IN {total_time:.2f} SECONDS")
    print("=" * 60)


Step 11: Computing LSTM OOF (expensive operation)
Computing LSTM OOF for 2079 sequences (this may take a while...)


/raid/invigoworks/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step0/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step0/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step0/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step0/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step0/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step30/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step40/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/ste

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step50/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step60/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/ste

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step70/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step80/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step90/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step00/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step10/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/st

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step20/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step30/2049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [3]:
import os, glob, re
import pandas as pd, numpy as np, yfinance as yf, warnings
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings('ignore')

ROOT_DIR = "./news_data"    
START_DATE = "2020-01-01"
END_DATE = "2025-09-23"
TRAIN_END = "2024-12-31"
TICKER = "ETH-USD"
SEQ_LEN = 10
np.random.seed(42); tf.random.set_seed(42)

def parse_date_from_filename(fname):
    bn = os.path.basename(fname)
    m = re.search(r'(\d{4})[^\d](\d{2})[^\d](\d{2})', bn)
    if m:
        return pd.to_datetime(f"{m.group(1)}-{m.group(2)}-{m.group(3)}").normalize()
    return None

def read_and_aggregate_folder(folder):
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    rows = []
    for f in files:
        try:
            df = pd.read_csv(f)
        except:
            continue
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date']).dt.normalize()
        else:
            dt = parse_date_from_filename(f)
            if dt is None:
                continue
            df['date'] = dt
        if 'label' not in df.columns:
            if 'sentiment' in df.columns:
                df['label'] = df['sentiment']
            else:
                continue
        df['label'] = pd.to_numeric(df['label'], errors='coerce').fillna(0).astype(int)
        if 'news' in df.columns:
            df['title_len'] = df['news'].astype(str).map(len)
        else:
            df['title_len'] = 0
        rows.append(df[['date','label','title_len']])
    if not rows:
        return pd.DataFrame(columns=['date','news_count','sum_sentiment','mean_sentiment','std_sentiment','pos_ratio','neg_ratio','neu_ratio','avg_title_len'])
    all_df = pd.concat(rows, ignore_index=True)
    agg = all_df.groupby('date').agg(
        news_count=('label','count'),
        sum_sentiment=('label','sum'),
        mean_sentiment=('label','mean'),
        std_sentiment=('label','std'),
        avg_title_len=('title_len','mean')
    )
    pos = all_df.groupby('date')['label'].apply(lambda s: (s==1).sum())
    neg = all_df.groupby('date')['label'].apply(lambda s: (s==-1).sum())
    neu = all_df.groupby('date')['label'].apply(lambda s: (s==0).sum())
    agg['pos_ratio'] = pos / agg['news_count']
    agg['neg_ratio'] = neg / agg['news_count']
    agg['neu_ratio'] = neu / agg['news_count']
    agg['std_sentiment'] = agg['std_sentiment'].fillna(0)
    return agg.reset_index()

def fetch_price(start_date, end_date, ticker=TICKER):
    df = yf.download(ticker, start=start_date, end=(pd.to_datetime(end_date)+pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False, auto_adjust=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns=df.columns.get_level_values(0)
    df = df.reset_index()
    df = df.rename(columns={'Date':'date'})
    df['date'] = pd.to_datetime(df['date']).dt.normalize()

    close_col = next((c for c in df.columns if 'close' in str(c).lower()), None)
    if close_col is None:
        raise RuntimeError("No close column in price data")
    df['close'] = df[close_col]
    df = df[['date','close']].copy()
    df = df[df['date'] >= pd.to_datetime(START_DATE)].reset_index(drop=True)
    return df

def make_sequences(arr, lookback):
    Xs=[]
    for i in range(lookback, len(arr)):
        Xs.append(arr[i-lookback:i])
    return np.array(Xs)

def build_lstm(input_dim, units=32):
    m = Sequential([LSTM(units, input_shape=(SEQ_LEN, input_dim)), Dropout(0.2), Dense(1, activation='sigmoid')])
    m.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return m

agg = read_and_aggregate_folder(ROOT_DIR)
agg = agg[(agg['date']>=pd.to_datetime(START_DATE)) & (agg['date']<=pd.to_datetime(END_DATE))].reset_index(drop=True)
for w in [3,7,14]:
    agg[f'roll_mean_sent_{w}'] = agg['sum_sentiment'].rolling(window=w, min_periods=1).mean()
    agg[f'roll_std_sent_{w}'] = agg['sum_sentiment'].rolling(window=w, min_periods=1).std().fillna(0)
    agg[f'roll_sum_count_{w}'] = agg['news_count'].rolling(window=w, min_periods=1).sum()
agg['sent_lag_1'] = agg['sum_sentiment'].shift(1).fillna(0)

price = fetch_price(START_DATE, END_DATE, TICKER)
df = pd.merge(price, agg, on='date', how='left').sort_values('date').reset_index(drop=True)
df[['news_count','sum_sentiment','mean_sentiment','std_sentiment','pos_ratio','neg_ratio','neu_ratio','avg_title_len']] = df[['news_count','sum_sentiment','mean_sentiment','std_sentiment','pos_ratio','neg_ratio','neu_ratio','avg_title_len']].fillna(0)

df['ret'] = df['close'].pct_change()
df['target'] = (df['ret'] > 0).astype(int)
for l in [1,2,3]:
    df[f'ret_lag_{l}'] = df['ret'].shift(l)

price_feats = [f'ret_lag_{i}' for i in [1,2,3]]
news_feats = ['sum_sentiment','mean_sentiment','std_sentiment','pos_ratio','neg_ratio','neu_ratio','news_count','avg_title_len',
              'roll_mean_sent_3','roll_std_sent_3','roll_sum_count_3',
              'roll_mean_sent_7','roll_std_sent_7','roll_sum_count_7',
              'roll_mean_sent_14','roll_std_sent_14','roll_sum_count_14','sent_lag_1']
news_feats = [c for c in news_feats if c in df.columns]
all_feats = price_feats + news_feats

data = df[['date'] + all_feats + ['target']].dropna().reset_index(drop=True)
train_mask = data['date'] <= pd.to_datetime(TRAIN_END)
test_mask = data['date'] > pd.to_datetime(TRAIN_END)
y_train = data.loc[train_mask,'target'].values; y_test = data.loc[test_mask,'target'].values
Xp_train = data.loc[train_mask, price_feats].values; Xp_test = data.loc[test_mask, price_feats].values
Xn_train = data.loc[train_mask, all_feats].values; Xn_test = data.loc[test_mask, all_feats].values

scaler_p = StandardScaler().fit(Xp_train); Xp_train_s = scaler_p.transform(Xp_train); Xp_test_s = scaler_p.transform(Xp_test)
scaler_n = StandardScaler().fit(Xn_train); Xn_train_s = scaler_n.transform(Xn_train); Xn_test_s = scaler_n.transform(Xn_test)

def train_classifiers(Xtr, ytr):
    res={}
    res['Logit'] = LogisticRegression(max_iter=2000).fit(Xtr, ytr)
    res['ANN'] = MLPClassifier(hidden_layer_sizes=(10,), max_iter=2000).fit(Xtr, ytr)
    res['SVM'] = SVC(kernel='rbf', probability=True).fit(Xtr, ytr)
    return res

res_price = train_classifiers(Xp_train_s, y_train)
res_news = train_classifiers(Xn_train_s, y_train)

full_price = data[price_feats].values
full_news = data[all_feats].values
Xp_seq = make_sequences(full_price, SEQ_LEN)
Xn_seq = make_sequences(full_news, SEQ_LEN)
y_seq = data['target'].values[SEQ_LEN:]
dates_seq = data['date'].values[SEQ_LEN:]
train_seq_mask = dates_seq <= np.datetime64(TRAIN_END)
test_seq_mask = dates_seq > np.datetime64(TRAIN_END)

def scale_seq(X_seq):
    n,s,f = X_seq.shape
    flat = X_seq.reshape(-1,f)
    sc = StandardScaler().fit(flat)
    scaled = sc.transform(flat).reshape(n,s,f)
    return scaled, sc

Xp_seq_s, _ = scale_seq(Xp_seq)
Xn_seq_s, _ = scale_seq(Xn_seq)

model_p_lstm = build_lstm(Xp_seq_s.shape[2], units=32)
model_p_lstm.fit(Xp_seq_s[train_seq_mask], y_seq[train_seq_mask], epochs=10, batch_size=64, verbose=0)
prob_p_lstm = model_p_lstm.predict(Xp_seq_s[test_seq_mask])[:,0]; pred_p_lstm = (prob_p_lstm>0.5).astype(int)

model_n_lstm = build_lstm(Xn_seq_s.shape[2], units=32)
model_n_lstm.fit(Xn_seq_s[train_seq_mask], y_seq[train_seq_mask], epochs=10, batch_size=64, verbose=0)
prob_n_lstm = model_n_lstm.predict(Xn_seq_s[test_seq_mask])[:,0]; pred_n_lstm = (prob_n_lstm>0.5).astype(int)

price_full_scaled = scaler_p.transform(data[price_feats].values)
news_full_scaled = scaler_n.transform(data[all_feats].values)
price_test_full = price_full_scaled[test_mask.values]
news_test_full = news_full_scaled[test_mask.values]

def compute_metrics(y_true, y_pred, y_prob=None):
    mask = ~pd.isna(y_pred)
    if mask.sum()==0:
        return {'Accuracy':np.nan,'Precision':np.nan,'Recall':np.nan,'F1':np.nan,'AUC':np.nan,'MDA':np.nan,'MDE':np.nan}
    y_t = y_true[mask].astype(int); y_p = y_pred[mask].astype(int)
    out={}
    out['Accuracy']=accuracy_score(y_t,y_p)
    out['Precision']=precision_score(y_t,y_p,zero_division=0)
    out['Recall']=recall_score(y_t,y_p,zero_division=0)
    out['F1']=f1_score(y_t,y_p,zero_division=0)
    out['AUC']=roc_auc_score(y_t,y_prob[mask]) if (y_prob is not None and len(np.unique(y_t))>1) else np.nan
    out['MDA']=out['Accuracy']; out['MDE']=np.mean(y_p != y_t)
    return out

rows=[]
for name, m in res_price.items():
    prob = m.predict_proba(price_test_full)[:,1]; pred = (prob>0.5).astype(int)
    rows.append({'Model':f'P_{name}', **compute_metrics(y_test, pred, prob)})

seq_global_idx = np.arange(SEQ_LEN, len(data))
seq_test_idx = seq_global_idx[dates_seq > np.datetime64(TRAIN_END)]
test_global_idx = np.where(test_mask.values)[0]
arr_pred = np.full(len(y_test), np.nan); arr_prob = np.full(len(y_test), np.nan)
for i, gidx in enumerate(seq_test_idx):
    pos = np.where(test_global_idx==gidx)[0]
    if pos.size>0:
        arr_pred[pos[0]] = pred_p_lstm[i]; arr_prob[pos[0]] = prob_p_lstm[i]
rows.append({'Model':'P_LSTM', **compute_metrics(y_test, arr_pred, arr_prob)})

for name, m in res_news.items():
    prob = m.predict_proba(news_test_full)[:,1]; pred = (prob>0.5).astype(int)
    rows.append({'Model':f'N_{name}', **compute_metrics(y_test, pred, prob)})

arr_pred_n = np.full(len(y_test), np.nan); arr_prob_n = np.full(len(y_test), np.nan)
for i, gidx in enumerate(seq_test_idx):
    pos = np.where(test_global_idx==gidx)[0]
    if pos.size>0:
        arr_pred_n[pos[0]] = pred_n_lstm[i]; arr_prob_n[pos[0]] = prob_n_lstm[i]
rows.append({'Model':'N_LSTM', **compute_metrics(y_test, arr_pred_n, arr_prob_n)})

full_probs=[]
for name, m in res_news.items():
    full_probs.append(m.predict_proba(news_full_scaled)[:,1])
lstm_full = np.concatenate([np.full(SEQ_LEN, np.nan), model_n_lstm.predict(Xn_seq_s)[:,0]])
full_probs.append(lstm_full)
full_probs = np.vstack(full_probs).T

seq_start = SEQ_LEN
train_indices = np.where(data['date'].values[seq_start:] <= np.datetime64(TRAIN_END))[0] + seq_start
test_indices  = np.where(data['date'].values[seq_start:] >  np.datetime64(TRAIN_END))[0] + seq_start
case_probs_train = full_probs[train_indices]; case_probs_test = full_probs[test_indices]
y_train_case = data['target'].values[train_indices]; y_test_case = data['target'].values[test_indices]

best_k, best_acc = 1, -1
for k in range(1,11):
    neigh = NearestNeighbors(n_neighbors=k).fit(case_probs_train)
    _, idxs = neigh.kneighbors(case_probs_test)
    preds = np.array([int(np.round(y_train_case[idxs[i]].mean())) for i in range(len(idxs))])
    acc = accuracy_score(y_test_case, preds)
    if acc > best_acc:
        best_acc = acc; best_k = k; best_preds = preds

cbr_full_pred = np.full(len(y_test), np.nan)
for i, gidx in enumerate(test_indices):
    pos = np.where(test_global_idx==gidx)[0]
    if pos.size>0:
        cbr_full_pred[pos[0]] = best_preds[i]
rows.append({'Model':'N_CBR', **compute_metrics(y_test, cbr_full_pred, None)})

res_df = pd.DataFrame(rows).set_index('Model').round(4)
print('data rows:', data.shape[0], 'train rows:', Xp_train.shape[0], 'test rows:', Xp_test.shape[0])
print(res_df.sort_values('Accuracy', ascending=False))

for idx in res_df.sort_values('Accuracy', ascending=False).index:
    print('\nModel:', idx)
    if idx == 'P_LSTM':
        ypred_arr = arr_pred
    elif idx == 'N_LSTM':
        ypred_arr = arr_pred_n
    elif idx == 'N_CBR':
        ypred_arr = cbr_full_pred
    elif idx.startswith('P_'):
        name = idx.split('_',1)[1]; ypred_arr = (res_price[name].predict(price_test_full)>0.5).astype(int)
    else:
        name = idx.split('_',1)[1]; ypred_arr = (res_news[name].predict(news_test_full)>0.5).astype(int)
    mask = ~pd.isna(ypred_arr)
    if mask.sum()>0:
        print('Confusion:\n', confusion_matrix(y_test[mask].astype(int), ypred_arr[mask].astype(int)))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
data rows: 2046 train rows: 1781 test rows: 265
         Accuracy  Precision  Recall      F1     AUC     MDA     MDE
Model                                                               
P_SVM      0.5358     0.5307  0.8832  0.6630  0.5164  0.5358  0.4642
P_Logit    0.5283     0.5273  0.8467  0.6499  0.5266  0.5283  0.4717
N_CBR      0.5245     0.5524  0.4234  0.4793     NaN  0.5245  0.4755
N_SVM      0.5094     0.5181  0.7299  0.6061  0.4764  0.5094  0.4906
P_ANN      0.4981     0.5101  0.7372  0.6030  0.4962  0.4981  0.5019
N_LSTM     0.4943     0.5082  0.6788  0.5812  0.4975  0.4943  0.5057
P_LSTM     0.4906     0.5043  0.8540  0.6341  0.4948  0.4906  0.5094
N_ANN      0.4906     0.5096  0.3869  0.4398  0.5145  0.4906  0.5094
N_Logit    0.4792     0.4954  0.3942  0.4390  0.4940  0.4792  0.5208

Model: P_SVM
Confusion:
 [[ 43  85]
 [ 35 102]]

Model: P_Logit
Confusion:
 [

In [19]:
import os, glob, re
import pandas as pd, numpy as np, yfinance as yf, warnings
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings('ignore')

ROOT_DIR = "./news_data"    
START_DATE = "2020-01-01"
END_DATE = "2025-09-23"
TRAIN_END = "2024-12-31"
TICKER = "ETH-USD"
SEQ_LEN = 3
np.random.seed(42); tf.random.set_seed(42)

# 한국어 암호화폐/금융 감성사전
POSITIVE_WORDS = {
    '좋다': 0.7, '좋은': 0.7, '훌륭': 0.8, '최고': 0.9, '대박': 0.9, '성공': 0.8,
    '상승': 0.8, '오르다': 0.8, '올라': 0.8, '급등': 0.9, '폭등': 0.9, '상한가': 0.95,
    '수익': 0.8, '이익': 0.8, '흑자': 0.8, '플러스': 0.7, '긍정': 0.7, '호재': 0.85,
    '강세': 0.8, '강하다': 0.7, '견고': 0.7, '탄탄': 0.8, '활발': 0.7,
    '투자': 0.6, '매수': 0.7, '불': 0.8, '뜨겁다': 0.7, '핫': 0.7,
    '성장': 0.8, '확대': 0.7, '증가': 0.8, '늘어': 0.7, '높다': 0.6,
    '돌파': 0.8, '넘어': 0.7, '뛰어': 0.8, '점프': 0.7, '도약': 0.8,
    '기대': 0.6, '희망': 0.7, '낙관': 0.8, '밝다': 0.7, '긍정적': 0.7,
    '호황': 0.8, '붐': 0.8, '열기': 0.7, '인기': 0.7, '관심': 0.6,
    '달': 0.9, '문': 0.9, '로켓': 0.8, '펌프': 0.7, '홀드': 0.6
}

NEGATIVE_WORDS = {
    '나쁘다': -0.7, '나쁜': -0.7, '최악': -0.9, '망': -0.8, '실패': -0.8,
    '하락': -0.8, '떨어지다': -0.8, '떨어져': -0.8, '급락': -0.9, '폭락': -0.9, '하한가': -0.95,
    '손실': -0.8, '적자': -0.8, '마이너스': -0.7, '부정': -0.7, '악재': -0.85,
    '약세': -0.8, '약하다': -0.7, '불안': -0.8, '위험': -0.8, '침체': -0.8,
    '매도': -0.7, '팔다': -0.7, '버리다': -0.8, '던지다': -0.8,
    '감소': -0.8, '줄어': -0.7, '낮다': -0.6, '밑': -0.6, '아래': -0.6,
    '붕괴': -0.9, '무너지다': -0.9, '크래시': -0.9, '폭망': -0.9,
    '우려': -0.7, '걱정': -0.7, '불안': -0.8, '어둡다': -0.7, '부정적': -0.7,
    '공포': -0.8, '패닉': -0.9, '두려움': -0.8, '무서': -0.7,
    '덤프': -0.8, '쇼트': -0.7, '베어': -0.8, '곰': -0.7, '약': -0.6
}

def calculate_sentiment_score(text):
    """한국어 뉴스 헤드라인 감성분석"""
    if pd.isna(text) or text == '':
        return 0.0
    
    text = str(text)
    scores = []
    
    # 한국어는 공백으로 분리하되, 부분 매칭도 고려
    for pos_word, pos_score in POSITIVE_WORDS.items():
        if pos_word in text:
            scores.append(pos_score)
    
    for neg_word, neg_score in NEGATIVE_WORDS.items():
        if neg_word in text:
            scores.append(neg_score)
    
    return np.mean(scores) if scores else 0.0

def classify_sentiment(score):
    """극성값 기준으로 분류"""
    if score > 0:
        return 1  # 긍정
    elif score < 0:
        return -1  # 부정
    else:
        return 0  # 중립

def parse_date_from_filename(fname):
    bn = os.path.basename(fname)
    m = re.search(r'(\d{4})[^\d](\d{2})[^\d](\d{2})', bn)
    if m:
        return pd.to_datetime(f"{m.group(1)}-{m.group(2)}-{m.group(3)}").normalize()
    return None

def read_and_aggregate_news_lexicon(folder):
    """논문 방식으로 뉴스 처리"""
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    rows = []
    
    for f in files:
        try:
            df = pd.read_csv(f)
        except:
            continue
            
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date']).dt.normalize()
        else:
            dt = parse_date_from_filename(f)
            if dt is None:
                continue
            df['date'] = dt
        
        # 뉴스 헤드라인에 대한 감성분석
        if 'news' in df.columns:
            df['sentiment_score'] = df['news'].apply(calculate_sentiment_score)
            df['sentiment_class'] = df['sentiment_score'].apply(classify_sentiment)
        else:
            continue
            
        rows.append(df[['date', 'sentiment_score', 'sentiment_class']])
    
    if not rows:
        return pd.DataFrame(columns=['date', 'total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio', 'avg_sentiment'])
    
    all_df = pd.concat(rows, ignore_index=True)
    
    # 일별 집계 (논문과 동일)
    daily_agg = all_df.groupby('date').agg(
        total_news=('sentiment_class', 'count'),
        avg_sentiment=('sentiment_score', 'mean'),
        pos_count=('sentiment_class', lambda x: (x == 1).sum()),
        neg_count=('sentiment_class', lambda x: (x == -1).sum()),
        neu_count=('sentiment_class', lambda x: (x == 0).sum())
    ).reset_index()
    
    # 긍정/부정 뉴스 비율 계산 (논문 핵심 피쳐)
    daily_agg['pos_ratio'] = daily_agg['pos_count'] / daily_agg['total_news']
    daily_agg['neg_ratio'] = daily_agg['neg_count'] / daily_agg['total_news']
    daily_agg['neu_ratio'] = daily_agg['neu_count'] / daily_agg['total_news']
    
    return daily_agg[['date', 'total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio', 'avg_sentiment']]

def fetch_price_data(start_date, end_date, ticker=TICKER):
    df = yf.download(ticker, start=start_date, end=(pd.to_datetime(end_date)+pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.reset_index()
    df = df.rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    
    close_col = next((c for c in df.columns if 'close' in str(c).lower()), None)
    if close_col is None:
        raise RuntimeError("No close column found")
    
    df['close'] = df[close_col]
    return df[['date', 'close']].copy()

def add_technical_indicators(df):
    """가격 데이터프레임에 기술적 지표를 추가하는 함수"""
    # 이동평균선 (SMA)
    df['SMA_12'] = df['close'].rolling(window=12).mean()
    df['SMA_26'] = df['close'].rolling(window=26).mean()

    # RSI (상대강도지수)
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD (이동평균 수렴확산)
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # 골든크로스 (1), 데드크로스 (0)
    df['SMA_cross'] = (df['SMA_12'] > df['SMA_26']).astype(int)
    
    # MACD 히스토그램 (MACD 값 - 시그널 값)
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    # RSI 과매수(1), 과매도(-1), 나머지(0)
    df['RSI_over'] = np.where(df['RSI'] > 70, 1, np.where(df['RSI'] < 30, -1, 0))
    
    return df

def make_lstm_dataset(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)


def build_lstm_model(input_dim, units=50):
    model = Sequential([
        LSTM(units, input_shape=(SEQ_LEN, input_dim)),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def make_sequences(data, seq_len):
    X = []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
    return np.array(X)

def compute_metrics(y_true, y_pred, y_prob=None):
    mask = ~pd.isna(y_pred)
    if mask.sum() == 0:
        return {'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1': np.nan, 'AUC': np.nan}
    
    y_t = y_true[mask].astype(int)
    y_p = y_pred[mask].astype(int)
    
    metrics = {
        'Accuracy': accuracy_score(y_t, y_p),
        'Precision': precision_score(y_t, y_p, zero_division=0),
        'Recall': recall_score(y_t, y_p, zero_division=0),
        'F1': f1_score(y_t, y_p, zero_division=0)
    }
    
    if y_prob is not None and len(np.unique(y_t)) > 1:
        metrics['AUC'] = roc_auc_score(y_t, y_prob[mask])
    else:
        metrics['AUC'] = np.nan
    
    return metrics

print("=== 한국어 어휘기반 감성분석 (논문 방식) ===")

# 1. 뉴스 데이터 처리 (논문 방식)
news_data = read_and_aggregate_news_lexicon(ROOT_DIR)
news_data = news_data[(news_data['date'] >= pd.to_datetime(START_DATE)) & 
                     (news_data['date'] <= pd.to_datetime(END_DATE))].reset_index(drop=True)

print(f"뉴스 데이터 크기: {news_data.shape}")
print(f"날짜 범위: {news_data['date'].min()} ~ {news_data['date'].max()}")
print(f"평균 긍정 비율: {news_data['pos_ratio'].mean():.3f}")
print(f"평균 부정 비율: {news_data['neg_ratio'].mean():.3f}")
print(f"평균 중립 비율: {news_data['neu_ratio'].mean():.3f}")

# 2. 가격 데이터 처리
price_data = fetch_price_data(START_DATE, END_DATE, TICKER)
price_data = price_data[price_data['date'] >= pd.to_datetime(START_DATE)].reset_index(drop=True)

price_data= add_technical_indicators(price_data)

# 3. 데이터 병합 및 타겟 생성
df = pd.merge(price_data, news_data, on='date', how='left').sort_values('date').reset_index(drop=True)

# 뉴스 없는 날은 0으로 채움
news_cols = ['total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio', 'avg_sentiment']
df[news_cols] = df[news_cols].fillna(0)

# 가격 변동률 및 타겟 생성
df['return'] = df['close'].pct_change()
df['target'] = (df['return'] > 0).astype(int)

# 논문에서 사용한 피쳐: 1일, 2일, 3일 전 가격변동률 + 1일 시차 뉴스
for lag in [1, 2, 3]:
    df[f'return_lag_{lag}'] = df['return'].shift(lag)
    df[f'pos_ratio_lag_{lag}'] = df['pos_ratio'].shift(lag)  # 1일 시차
    df[f'neg_ratio_lag_{lag}'] = df['neg_ratio'].shift(lag)  # 1일 시차
    df[f'avg_sentiment_lag_{lag}'] = df['avg_sentiment'].shift(lag)  # 1일 시차

# 피쳐 정의 (논문 방식)
price_features = ['return_lag_1', 'return_lag_2', 'return_lag_3','SMA_12',
                  'SMA_26','RSI','MACD', 'MACD_signal','SMA_cross','MACD_hist','RSI_over']
news_features = ['pos_ratio_lag_1', 'neg_ratio_lag_1', 'avg_sentiment_lag_1']  # 1일 시차만
# 뉴스 감성 피처 추가
df['pos_x_ret'] = df['pos_ratio_lag_1'] * df['return_lag_1']
df['neg_x_ret'] = df['neg_ratio_lag_1'] * df['return_lag_1']
df['sentiment_x_ret'] = df['avg_sentiment_lag_1'] * df['return_lag_1']
df['total_news_lag_1'] = df['total_news'].shift(1)
df['sentiment_x_volume'] = df['avg_sentiment_lag_1'] * df['total_news_lag_1']


combined_features = price_features + news_features + ['pos_x_ret', 'neg_x_ret', 'sentiment_x_ret','sentiment_x_volume']

threshold = 0.005 # 0.5%
df['target'] = np.where(df['return'] > threshold, 1, np.where(df['return'] < -threshold, 0, np.nan))

# 데이터 준비
data_clean = df[['date'] + combined_features + ['target']].dropna().reset_index(drop=True)
print(f"Clean data shape: {data_clean.shape}")

# 학습/테스트 분할
train_mask = data_clean['date'] <= pd.to_datetime(TRAIN_END)
test_mask = data_clean['date'] > pd.to_datetime(TRAIN_END)

X_price_train = data_clean.loc[train_mask, price_features].values
X_price_test = data_clean.loc[test_mask, price_features].values
X_news_train = data_clean.loc[train_mask, news_features].values
X_news_test = data_clean.loc[test_mask, news_features].values
X_combined_train = data_clean.loc[train_mask, combined_features].values
X_combined_test = data_clean.loc[test_mask, combined_features].values

y_train = data_clean.loc[train_mask, 'target'].values
y_test = data_clean.loc[test_mask, 'target'].values

print(f"학습 샘플: {len(y_train)}, 테스트 샘플: {len(y_test)}")
print(f"가격 피쳐: {price_features}")
print(f"뉴스 피쳐: {news_features}")

# 감성분석 품질 체크
sentiment_sample = data_clean['avg_sentiment_lag_1']
print(f"\n감성 점수 분포:")
print(f"평균: {sentiment_sample.mean():.4f}")
print(f"표준편차: {sentiment_sample.std():.4f}")
print(f"긍정(>0): {(sentiment_sample > 0).sum()}/{len(sentiment_sample)}")
print(f"부정(<0): {(sentiment_sample < 0).sum()}/{len(sentiment_sample)}")
print(f"중립(=0): {(sentiment_sample == 0).sum()}/{len(sentiment_sample)}")

# 스케일링
scaler_p = StandardScaler().fit(X_price_train)
X_price_train_scaled = scaler_p.transform(X_price_train)
X_price_test_scaled = scaler_p.transform(X_price_test)

scaler_n = StandardScaler().fit(X_news_train)
X_news_train_scaled = scaler_n.transform(X_news_train)
X_news_test_scaled = scaler_n.transform(X_news_test)

scaler_c = StandardScaler().fit(X_combined_train)
X_combined_train_scaled = scaler_c.transform(X_combined_train)
X_combined_test_scaled = scaler_c.transform(X_combined_test)

# 모델 학습 및 평가
def train_models(X_train, X_test, y_train, y_test, model_prefix):
    results = []
    
    # 로지스틱 회귀
    logit = LogisticRegression(max_iter=2000, random_state=42)
    logit.fit(X_train, y_train)
    pred_logit = logit.predict(X_test)
    prob_logit = logit.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_Logit', **compute_metrics(y_test, pred_logit, prob_logit)})
    
    # 인공신경망
    ann = MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=2000, random_state=42)
    ann.fit(X_train, y_train)
    pred_ann = ann.predict(X_test)
    prob_ann = ann.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_ANN', **compute_metrics(y_test, pred_ann, prob_ann)})
    
    # SVM
    svm = SVC(kernel='rbf', probability=True, random_state=42)
    svm.fit(X_train, y_train)
    pred_svm = svm.predict(X_test)
    prob_svm = svm.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_SVM', **compute_metrics(y_test, pred_svm, prob_svm)})
    
    return results

# LSTM 모델 (시퀀스 데이터용)
def train_lstm_model(X_train, X_test, y_train, y_test, model_name, seq_len=SEQ_LEN):
    if len(X_train) < seq_len:
        return {'Model': model_name, 'Accuracy': np.nan, 'Note': 'Insufficient data for LSTM'}
    
    # 시퀀스 생성
    X_train_seq, y_train_seq = make_lstm_dataset(X_train, y_train, seq_len)
    X_test_seq, y_test_seq = make_lstm_dataset(X_test, y_test, seq_len)

    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        return {'Model': model_name, 'Accuracy': np.nan, 'Note': 'No sequences generated'}
    
    # LSTM 모델 구축 및 학습
    model = build_lstm_model(X_train_seq.shape[2])
    model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=32, verbose=0)
    
    # 예측
    prob_lstm = model.predict(X_test_seq, verbose=0)[:, 0]
    pred_lstm = (prob_lstm > 0.5).astype(int)
    
    return {'Model': model_name, **compute_metrics(y_test_seq, pred_lstm, prob_lstm)}

print("\n=== 모델 학습 및 평가 ===")

# 1. 가격 기반 모델 (P_)
price_results = train_models(X_price_train_scaled, X_price_test_scaled, y_train, y_test, 'P')

# 2. 뉴스 기반 모델 (N_)
news_results = train_models(X_news_train_scaled, X_news_test_scaled, y_train, y_test, 'N')

# 3. 결합 모델 (Combined_)
combined_results = train_models(X_combined_train_scaled, X_combined_test_scaled, y_train, y_test, 'Combined')

# 4. LSTM 모델들
lstm_price = train_lstm_model(X_price_train_scaled, X_price_test_scaled, y_train, y_test, 'P_LSTM')
lstm_news = train_lstm_model(X_news_train_scaled, X_news_test_scaled, y_train, y_test, 'N_LSTM')
lstm_combined = train_lstm_model(X_combined_train_scaled, X_combined_test_scaled, y_train, y_test, 'Combined_LSTM')

# 결과 정리
all_results = price_results + news_results + combined_results
if not pd.isna(lstm_price.get('Accuracy')):
    all_results.append(lstm_price)
if not pd.isna(lstm_news.get('Accuracy')):
    all_results.append(lstm_news)
if not pd.isna(lstm_combined.get('Accuracy')):
    all_results.append(lstm_combined)

results_df = pd.DataFrame(all_results).set_index('Model').round(4)

print("=== 최종 성능 비교 ===")
print(results_df.sort_values('Accuracy', ascending=False))

# 성능 개선 분석
price_best = results_df[results_df.index.str.startswith('P_')]['Accuracy'].max()
news_best = results_df[results_df.index.str.startswith('N_')]['Accuracy'].max()
combined_best = results_df[results_df.index.str.startswith('Combined_')]['Accuracy'].max()

print(f"\n=== 성능 개선 분석 ===")
print(f"최고 가격 모델: {price_best:.4f}")
print(f"최고 뉴스 모델: {news_best:.4f}")
print(f"최고 결합 모델: {combined_best:.4f}")
print(f"뉴스 추가 효과: {(combined_best - price_best) * 100:.2f}%")

# 감성-수익률 상관관계 분석
sentiment_corr = np.corrcoef(data_clean['avg_sentiment_lag_1'], data_clean['target'])[0, 1]
print(f"감성-수익률 상관계수: {sentiment_corr:.4f}")

if combined_best > price_best:
    print("✓ 뉴스 감성정보가 예측성능을 향상시켰습니다!")
else:
    print("✗ 뉴스 감성정보가 예측성능을 저하시켰습니다.")
    print("개선방안: 감성사전 확장, 가중치 조정, 시간지연 최적화 필요")

=== 한국어 어휘기반 감성분석 (논문 방식) ===
뉴스 데이터 크기: (2050, 6)
날짜 범위: 2020-01-01 00:00:00 ~ 2025-09-22 00:00:00
평균 긍정 비율: 0.794
평균 부정 비율: 0.086
평균 중립 비율: 0.120
Clean data shape: (1758, 20)
학습 샘플: 1540, 테스트 샘플: 218
가격 피쳐: ['return_lag_1', 'return_lag_2', 'return_lag_3', 'SMA_12', 'SMA_26', 'RSI', 'MACD', 'MACD_signal', 'SMA_cross', 'MACD_hist', 'RSI_over']
뉴스 피쳐: ['pos_ratio_lag_1', 'neg_ratio_lag_1', 'avg_sentiment_lag_1']

감성 점수 분포:
평균: 0.3246
표준편차: 0.1613
긍정(>0): 1677/1758
부정(<0): 34/1758
중립(=0): 47/1758

=== 모델 학습 및 평가 ===
=== 최종 성능 비교 ===
                Accuracy  Precision  Recall      F1     AUC
Model                                                      
P_Logit           0.6514     0.6299  0.7339  0.6780  0.6873
P_SVM             0.6422     0.6348  0.6697  0.6518  0.6735
Combined_SVM      0.6376     0.6190  0.7156  0.6638  0.6566
Combined_Logit    0.6284     0.6207  0.6606  0.6400  0.6892
P_LSTM            0.5953     0.6044  0.5189  0.5584  0.6168
N_ANN             0.5734     0.5889  0.4862

In [4]:
import os, glob, re
import pandas as pd, numpy as np, yfinance as yf, warnings
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings('ignore')

ROOT_DIR = "./news_data"    
START_DATE = "2020-01-01"
END_DATE = "2025-09-23"
TRAIN_END = "2024-12-31"
TICKER = "ETH-USD"
SEQ_LEN = 3
np.random.seed(42); tf.random.set_seed(42)

def read_and_aggregate_news_label(folder):
    """라벨 컬럼 기반 뉴스 집계"""
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    rows = []

    for f in files:
        try:
            df = pd.read_csv(f)
        except:
            continue
        
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date']).dt.normalize()
        else:
            dt = parse_date_from_filename(f)
            if dt is None:
                continue
            df['date'] = dt
        
        if 'label' in df.columns:
            df['sentiment_class'] = df['label']  # 기존 label 그대로 사용
        else:
            continue

        rows.append(df[['date', 'sentiment_class']])
    
    if not rows:
        return pd.DataFrame(columns=['date', 'total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio'])
    
    all_df = pd.concat(rows, ignore_index=True)
    
    # 일별 집계
    daily_agg = all_df.groupby('date').agg(
        total_news=('sentiment_class', 'count'),
        pos_count=('sentiment_class', lambda x: (x == 1).sum()),
        neg_count=('sentiment_class', lambda x: (x == -1).sum()),
        neu_count=('sentiment_class', lambda x: (x == 0).sum())
    ).reset_index()
    
    daily_agg['pos_ratio'] = daily_agg['pos_count'] / daily_agg['total_news']
    daily_agg['neg_ratio'] = daily_agg['neg_count'] / daily_agg['total_news']
    daily_agg['neu_ratio'] = daily_agg['neu_count'] / daily_agg['total_news']

    daily_agg['avg_sentiment'] = all_df.groupby('date')['sentiment_class'].mean().values
    
    return daily_agg[['date', 'total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio','avg_sentiment']]

def fetch_price_data(start_date, end_date, ticker=TICKER):
    df = yf.download(ticker, start=start_date, end=(pd.to_datetime(end_date)+pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.reset_index()
    df = df.rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    
    close_col = next((c for c in df.columns if 'close' in str(c).lower()), None)
    if close_col is None:
        raise RuntimeError("No close column found")
    
    df['close'] = df[close_col]
    return df[['date', 'close']].copy()

def add_technical_indicators(df):
    """가격 데이터프레임에 기술적 지표를 추가하는 함수"""
    # 이동평균선 (SMA)
    df['SMA_12'] = df['close'].rolling(window=12).mean()
    df['SMA_26'] = df['close'].rolling(window=26).mean()

    # RSI (상대강도지수)
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD (이동평균 수렴확산)
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # 골든크로스 (1), 데드크로스 (0)
    df['SMA_cross'] = (df['SMA_12'] > df['SMA_26']).astype(int)
    
    # MACD 히스토그램 (MACD 값 - 시그널 값)
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    # RSI 과매수(1), 과매도(-1), 나머지(0)
    df['RSI_over'] = np.where(df['RSI'] > 70, 1, np.where(df['RSI'] < 30, -1, 0))
    
    return df

def make_lstm_dataset(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)


def build_lstm_model(input_dim, units=50):
    model = Sequential([
        LSTM(units, input_shape=(SEQ_LEN, input_dim)),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def make_sequences(data, seq_len):
    X = []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
    return np.array(X)

def compute_metrics(y_true, y_pred, y_prob=None):
    mask = ~pd.isna(y_pred)
    if mask.sum() == 0:
        return {'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1': np.nan, 'AUC': np.nan}
    
    y_t = y_true[mask].astype(int)
    y_p = y_pred[mask].astype(int)
    
    metrics = {
        'Accuracy': accuracy_score(y_t, y_p),
        'Precision': precision_score(y_t, y_p, zero_division=0),
        'Recall': recall_score(y_t, y_p, zero_division=0),
        'F1': f1_score(y_t, y_p, zero_division=0)
    }
    
    if y_prob is not None and len(np.unique(y_t)) > 1:
        metrics['AUC'] = roc_auc_score(y_t, y_prob[mask])
    else:
        metrics['AUC'] = np.nan
    
    return metrics

print("=== 한국어 어휘기반 감성분석 (논문 방식) ===")

# 1. 뉴스 데이터 처리 (논문 방식)
news_data = read_and_aggregate_news_label(ROOT_DIR)
news_data = news_data[(news_data['date'] >= pd.to_datetime(START_DATE)) & 
                     (news_data['date'] <= pd.to_datetime(END_DATE))].reset_index(drop=True)

print(f"뉴스 데이터 크기: {news_data.shape}")
print(f"날짜 범위: {news_data['date'].min()} ~ {news_data['date'].max()}")
print(f"평균 긍정 비율: {news_data['pos_ratio'].mean():.3f}")
print(f"평균 부정 비율: {news_data['neg_ratio'].mean():.3f}")
print(f"평균 중립 비율: {news_data['neu_ratio'].mean():.3f}")

# 2. 가격 데이터 처리
price_data = fetch_price_data(START_DATE, END_DATE, TICKER)
price_data = price_data[price_data['date'] >= pd.to_datetime(START_DATE)].reset_index(drop=True)

price_data= add_technical_indicators(price_data)

# 3. 데이터 병합 및 타겟 생성
df = pd.merge(price_data, news_data, on='date', how='left').sort_values('date').reset_index(drop=True)

# 뉴스 없는 날은 0으로 채움
news_cols = ['total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio', 'avg_sentiment']
df[news_cols] = df[news_cols].fillna(0)

# 가격 변동률 및 타겟 생성
df['return'] = df['close'].pct_change()
df['target'] = (df['return'] > 0).astype(int)

# 논문에서 사용한 피쳐: 1일, 2일, 3일 전 가격변동률 + 1일 시차 뉴스
for lag in [1, 2, 3]:
    df[f'return_lag_{lag}'] = df['return'].shift(lag)
    df[f'pos_ratio_lag_{lag}'] = df['pos_ratio'].shift(lag)  # 1일 시차
    df[f'neg_ratio_lag_{lag}'] = df['neg_ratio'].shift(lag)  # 1일 시차
    df[f'avg_sentiment_lag_{lag}'] = df['avg_sentiment'].shift(lag)  # 1일 시차

# 피쳐 정의 (논문 방식)
price_features = ['return_lag_1', 'return_lag_2', 'return_lag_3','SMA_12',
                  'SMA_26','RSI','MACD', 'MACD_signal','SMA_cross','MACD_hist','RSI_over']
news_features = ['pos_ratio_lag_1', 'neg_ratio_lag_1', 'avg_sentiment_lag_1']  # 1일 시차만
# 뉴스 감성 피처 추가
df['pos_x_ret'] = df['pos_ratio_lag_1'] * df['return_lag_1']
df['neg_x_ret'] = df['neg_ratio_lag_1'] * df['return_lag_1']
df['sentiment_x_ret'] = df['avg_sentiment_lag_1'] * df['return_lag_1']
df['total_news_lag_1'] = df['total_news'].shift(1)
df['sentiment_x_volume'] = df['avg_sentiment_lag_1'] * df['total_news_lag_1']


combined_features = price_features + news_features + ['pos_x_ret', 'neg_x_ret', 'sentiment_x_ret','sentiment_x_volume']

threshold = 0.005 # 0.5%
df['target'] = np.where(df['return'] > threshold, 1, np.where(df['return'] < -threshold, 0, np.nan))

# 데이터 준비
data_clean = df[['date'] + combined_features + ['target']].dropna().reset_index(drop=True)
print(f"Clean data shape: {data_clean.shape}")

# 학습/테스트 분할
train_mask = data_clean['date'] <= pd.to_datetime(TRAIN_END)
test_mask = data_clean['date'] > pd.to_datetime(TRAIN_END)

X_price_train = data_clean.loc[train_mask, price_features].values
X_price_test = data_clean.loc[test_mask, price_features].values
X_news_train = data_clean.loc[train_mask, news_features].values
X_news_test = data_clean.loc[test_mask, news_features].values
X_combined_train = data_clean.loc[train_mask, combined_features].values
X_combined_test = data_clean.loc[test_mask, combined_features].values

y_train = data_clean.loc[train_mask, 'target'].values
y_test = data_clean.loc[test_mask, 'target'].values

print(f"학습 샘플: {len(y_train)}, 테스트 샘플: {len(y_test)}")
print(f"가격 피쳐: {price_features}")
print(f"뉴스 피쳐: {news_features}")

# 감성분석 품질 체크
sentiment_sample = data_clean['avg_sentiment_lag_1']
print(f"\n감성 점수 분포:")
print(f"평균: {sentiment_sample.mean():.4f}")
print(f"표준편차: {sentiment_sample.std():.4f}")
print(f"긍정(>0): {(sentiment_sample > 0).sum()}/{len(sentiment_sample)}")
print(f"부정(<0): {(sentiment_sample < 0).sum()}/{len(sentiment_sample)}")
print(f"중립(=0): {(sentiment_sample == 0).sum()}/{len(sentiment_sample)}")

# 스케일링
scaler_p = StandardScaler().fit(X_price_train)
X_price_train_scaled = scaler_p.transform(X_price_train)
X_price_test_scaled = scaler_p.transform(X_price_test)

scaler_n = StandardScaler().fit(X_news_train)
X_news_train_scaled = scaler_n.transform(X_news_train)
X_news_test_scaled = scaler_n.transform(X_news_test)

scaler_c = StandardScaler().fit(X_combined_train)
X_combined_train_scaled = scaler_c.transform(X_combined_train)
X_combined_test_scaled = scaler_c.transform(X_combined_test)

# 모델 학습 및 평가
def train_models(X_train, X_test, y_train, y_test, model_prefix):
    results = []
    
    # 로지스틱 회귀
    logit = LogisticRegression(max_iter=2000, random_state=42)
    logit.fit(X_train, y_train)
    pred_logit = logit.predict(X_test)
    prob_logit = logit.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_Logit', **compute_metrics(y_test, pred_logit, prob_logit)})
    
    # 인공신경망
    ann = MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=2000, random_state=42)
    ann.fit(X_train, y_train)
    pred_ann = ann.predict(X_test)
    prob_ann = ann.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_ANN', **compute_metrics(y_test, pred_ann, prob_ann)})
    
    # SVM
    svm = SVC(kernel='rbf', probability=True, random_state=42)
    svm.fit(X_train, y_train)
    pred_svm = svm.predict(X_test)
    prob_svm = svm.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_SVM', **compute_metrics(y_test, pred_svm, prob_svm)})
    
    return results

# LSTM 모델 (시퀀스 데이터용)
def train_lstm_model(X_train, X_test, y_train, y_test, model_name, seq_len=SEQ_LEN):
    if len(X_train) < seq_len:
        return {'Model': model_name, 'Accuracy': np.nan, 'Note': 'Insufficient data for LSTM'}
    
    # 시퀀스 생성
    X_train_seq, y_train_seq = make_lstm_dataset(X_train, y_train, seq_len)
    X_test_seq, y_test_seq = make_lstm_dataset(X_test, y_test, seq_len)

    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        return {'Model': model_name, 'Accuracy': np.nan, 'Note': 'No sequences generated'}
    
    # LSTM 모델 구축 및 학습
    model = build_lstm_model(X_train_seq.shape[2])
    model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=32, verbose=0)
    
    # 예측
    prob_lstm = model.predict(X_test_seq, verbose=0)[:, 0]
    pred_lstm = (prob_lstm > 0.5).astype(int)
    
    return {'Model': model_name, **compute_metrics(y_test_seq, pred_lstm, prob_lstm)}

print("\n=== 모델 학습 및 평가 ===")

# 1. 가격 기반 모델 (P_)
price_results = train_models(X_price_train_scaled, X_price_test_scaled, y_train, y_test, 'P')

# 2. 뉴스 기반 모델 (N_)
news_results = train_models(X_news_train_scaled, X_news_test_scaled, y_train, y_test, 'N')

# 3. 결합 모델 (Combined_)
combined_results = train_models(X_combined_train_scaled, X_combined_test_scaled, y_train, y_test, 'Combined')

# 4. LSTM 모델들
lstm_price = train_lstm_model(X_price_train_scaled, X_price_test_scaled, y_train, y_test, 'P_LSTM')
lstm_news = train_lstm_model(X_news_train_scaled, X_news_test_scaled, y_train, y_test, 'N_LSTM')
lstm_combined = train_lstm_model(X_combined_train_scaled, X_combined_test_scaled, y_train, y_test, 'Combined_LSTM')

# 결과 정리
all_results = price_results + news_results + combined_results
if not pd.isna(lstm_price.get('Accuracy')):
    all_results.append(lstm_price)
if not pd.isna(lstm_news.get('Accuracy')):
    all_results.append(lstm_news)
if not pd.isna(lstm_combined.get('Accuracy')):
    all_results.append(lstm_combined)

results_df = pd.DataFrame(all_results).set_index('Model').round(4)

print("=== 최종 성능 비교 ===")
print(results_df.sort_values('Accuracy', ascending=False))

# 성능 개선 분석
price_best = results_df[results_df.index.str.startswith('P_')]['Accuracy'].max()
news_best = results_df[results_df.index.str.startswith('N_')]['Accuracy'].max()
combined_best = results_df[results_df.index.str.startswith('Combined_')]['Accuracy'].max()

print(f"\n=== 성능 개선 분석 ===")
print(f"최고 가격 모델: {price_best:.4f}")
print(f"최고 뉴스 모델: {news_best:.4f}")
print(f"최고 결합 모델: {combined_best:.4f}")
print(f"뉴스 추가 효과: {(combined_best - price_best) * 100:.2f}%")

# 감성-수익률 상관관계 분석
sentiment_corr = np.corrcoef(data_clean['avg_sentiment_lag_1'], data_clean['target'])[0, 1]
print(f"감성-수익률 상관계수: {sentiment_corr:.4f}")

if combined_best > price_best:
    print("✓ 뉴스 감성정보가 예측성능을 향상시켰습니다!")
else:
    print("✗ 뉴스 감성정보가 예측성능을 저하시켰습니다.")

=== 한국어 어휘기반 감성분석 (논문 방식) ===
뉴스 데이터 크기: (2050, 6)
날짜 범위: 2020-01-01 00:00:00 ~ 2025-09-22 00:00:00
평균 긍정 비율: 0.478
평균 부정 비율: 0.219
평균 중립 비율: 0.302
Clean data shape: (1758, 20)
학습 샘플: 1540, 테스트 샘플: 218
가격 피쳐: ['return_lag_1', 'return_lag_2', 'return_lag_3', 'SMA_12', 'SMA_26', 'RSI', 'MACD', 'MACD_signal', 'SMA_cross', 'MACD_hist', 'RSI_over']
뉴스 피쳐: ['pos_ratio_lag_1', 'neg_ratio_lag_1', 'avg_sentiment_lag_1']

감성 점수 분포:
평균: 0.2577
표준편차: 0.4322
긍정(>0): 1218/1758
부정(<0): 319/1758
중립(=0): 221/1758

=== 모델 학습 및 평가 ===


2025-09-30 13:19:30.473444: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46689 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1d:00.0, compute capability: 8.6
2025-09-30 13:19:36.165972: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


=== 최종 성능 비교 ===
                Accuracy  Precision  Recall      F1     AUC
Model                                                      
P_Logit           0.6514     0.6299  0.7339  0.6780  0.6873
P_SVM             0.6422     0.6348  0.6697  0.6518  0.6735
Combined_SVM      0.6330     0.6465  0.5872  0.6154  0.6497
Combined_Logit    0.6239     0.6116  0.6789  0.6435  0.6848
P_LSTM            0.5860     0.5859  0.5472  0.5659  0.6270
Combined_ANN      0.5688     0.5676  0.5780  0.5727  0.5972
Combined_LSTM     0.5581     0.5470  0.6038  0.5740  0.5945
P_ANN             0.5229     0.5210  0.5688  0.5439  0.5468
N_ANN             0.5183     0.5152  0.6239  0.5643  0.5208
N_Logit           0.5000     0.5000  1.0000  0.6667  0.5392
N_LSTM            0.4837     0.4877  0.9340  0.6408  0.5400
N_SVM             0.4817     0.4881  0.7523  0.5921  0.5487

=== 성능 개선 분석 ===
최고 가격 모델: 0.6514
최고 뉴스 모델: 0.5183
최고 결합 모델: 0.6330
뉴스 추가 효과: -1.84%
감성-수익률 상관계수: 0.0275
✗ 뉴스 감성정보가 예측성능을 저하시켰습니다.


In [2]:
import os, glob, re
import pandas as pd, numpy as np, yfinance as yf, warnings
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings('ignore')

ROOT_DIR = "./news_data"    
START_DATE = "2021-01-01"
END_DATE = "2025-10-02"
TRAIN_END = "2025-05-01"
TICKER = "ETH-USD"
SEQ_LEN = 3
np.random.seed(42); tf.random.set_seed(42)

def read_and_aggregate_news_label(folder):
    """라벨 컬럼 기반 뉴스 집계"""
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    rows = []

    for f in files:
        try:
            df = pd.read_csv(f)
        except:
            continue
        
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date']).dt.normalize()
        else:
            dt = parse_date_from_filename(f)
            if dt is None:
                continue
            df['date'] = dt
        
        if 'label' in df.columns:
            df['sentiment_class'] = df['label']  # 기존 label 그대로 사용
        else:
            continue

        rows.append(df[['date', 'sentiment_class']])
    
    if not rows:
        return pd.DataFrame(columns=['date', 'total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio'])
    
    all_df = pd.concat(rows, ignore_index=True)
    
    # 일별 집계
    daily_agg = all_df.groupby('date').agg(
        total_news=('sentiment_class', 'count'),
        pos_count=('sentiment_class', lambda x: (x == 1).sum()),
        neg_count=('sentiment_class', lambda x: (x == -1).sum()),
        neu_count=('sentiment_class', lambda x: (x == 0).sum())
    ).reset_index()
    
    daily_agg['pos_ratio'] = daily_agg['pos_count'] / daily_agg['total_news']
    daily_agg['neg_ratio'] = daily_agg['neg_count'] / daily_agg['total_news']
    daily_agg['neu_ratio'] = daily_agg['neu_count'] / daily_agg['total_news']

    daily_agg['avg_sentiment'] = all_df.groupby('date')['sentiment_class'].mean().values
    
    return daily_agg[['date', 'total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio','avg_sentiment']]

def fetch_price_data(start_date, end_date, ticker=TICKER):
    df = yf.download(ticker, start=start_date, end=(pd.to_datetime(end_date)+pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.reset_index()
    df = df.rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date']).dt.normalize()
    
    close_col = next((c for c in df.columns if 'close' in str(c).lower()), None)
    if close_col is None:
        raise RuntimeError("No close column found")
    
    df['close'] = df[close_col]
    return df[['date', 'close']].copy()

def add_technical_indicators(df):
    """가격 데이터프레임에 기술적 지표를 추가하는 함수"""
    # 이동평균선 (SMA)
    df['SMA_12'] = df['close'].rolling(window=12).mean()
    df['SMA_26'] = df['close'].rolling(window=26).mean()

    # RSI (상대강도지수)
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD (이동평균 수렴확산)
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # 골든크로스 (1), 데드크로스 (0)
    df['SMA_cross'] = (df['SMA_12'] > df['SMA_26']).astype(int)
    
    # MACD 히스토그램 (MACD 값 - 시그널 값)
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    # RSI 과매수(1), 과매도(-1), 나머지(0)
    df['RSI_over'] = np.where(df['RSI'] > 70, 1, np.where(df['RSI'] < 30, -1, 0))
    
    return df

def make_lstm_dataset(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)


def build_lstm_model(input_dim, units=50):
    model = Sequential([
        LSTM(units, input_shape=(SEQ_LEN, input_dim)),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def make_sequences(data, seq_len):
    X = []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i])
    return np.array(X)

def compute_metrics(y_true, y_pred, y_prob=None):
    mask = ~pd.isna(y_pred)
    if mask.sum() == 0:
        return {'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1': np.nan, 'AUC': np.nan}
    
    y_t = y_true[mask].astype(int)
    y_p = y_pred[mask].astype(int)
    
    metrics = {
        'Accuracy': accuracy_score(y_t, y_p),
        'Precision': precision_score(y_t, y_p, zero_division=0),
        'Recall': recall_score(y_t, y_p, zero_division=0),
        'F1': f1_score(y_t, y_p, zero_division=0)
    }
    
    if y_prob is not None and len(np.unique(y_t)) > 1:
        metrics['AUC'] = roc_auc_score(y_t, y_prob[mask])
    else:
        metrics['AUC'] = np.nan
    
    return metrics

print("=== 한국어 어휘기반 감성분석 (논문 방식) ===")

# 1. 뉴스 데이터 처리 (논문 방식)
news_data = read_and_aggregate_news_label(ROOT_DIR)
news_data = news_data[(news_data['date'] >= pd.to_datetime(START_DATE)) & 
                     (news_data['date'] <= pd.to_datetime(END_DATE))].reset_index(drop=True)

print(f"뉴스 데이터 크기: {news_data.shape}")
print(f"날짜 범위: {news_data['date'].min()} ~ {news_data['date'].max()}")
print(f"평균 긍정 비율: {news_data['pos_ratio'].mean():.3f}")
print(f"평균 부정 비율: {news_data['neg_ratio'].mean():.3f}")
print(f"평균 중립 비율: {news_data['neu_ratio'].mean():.3f}")

# 2. 가격 데이터 처리
price_data = fetch_price_data(START_DATE, END_DATE, TICKER)
price_data = price_data[price_data['date'] >= pd.to_datetime(START_DATE)].reset_index(drop=True)

price_data= add_technical_indicators(price_data)

# 3. 데이터 병합 및 타겟 생성
df = pd.merge(price_data, news_data, on='date', how='left').sort_values('date').reset_index(drop=True)

# 뉴스 없는 날은 0으로 채움
news_cols = ['total_news', 'pos_ratio', 'neg_ratio', 'neu_ratio', 'avg_sentiment']
df[news_cols] = df[news_cols].fillna(0)

# 가격 변동률 및 타겟 생성
df['return'] = df['close'].pct_change()
df['target'] = (df['return'] > 0).astype(int)

# 논문에서 사용한 피쳐: 1일, 2일, 3일 전 가격변동률 + 1일 시차 뉴스
for lag in [1, 2, 3]:
    df[f'return_lag_{lag}'] = df['return'].shift(lag)
    df[f'pos_ratio_lag_{lag}'] = df['pos_ratio'].shift(lag)  # 1일 시차
    df[f'neg_ratio_lag_{lag}'] = df['neg_ratio'].shift(lag)  # 1일 시차
    df[f'avg_sentiment_lag_{lag}'] = df['avg_sentiment'].shift(lag)  # 1일 시차

# 피쳐 정의 (논문 방식)
price_features = ['return_lag_1', 'return_lag_2', 'return_lag_3','SMA_12',
                  'SMA_26','RSI','MACD', 'MACD_signal','SMA_cross','MACD_hist','RSI_over']
news_features = ['pos_ratio_lag_1', 'neg_ratio_lag_1', 'avg_sentiment_lag_1']  # 1일 시차만
# 뉴스 감성 피처 추가
df['pos_x_ret'] = df['pos_ratio_lag_1'] * df['return_lag_1']
df['neg_x_ret'] = df['neg_ratio_lag_1'] * df['return_lag_1']
df['sentiment_x_ret'] = df['avg_sentiment_lag_1'] * df['return_lag_1']
df['total_news_lag_1'] = df['total_news'].shift(1)
df['sentiment_x_volume'] = df['avg_sentiment_lag_1'] * df['total_news_lag_1']


combined_features = price_features + news_features + ['pos_x_ret', 'neg_x_ret', 'sentiment_x_ret','sentiment_x_volume']

threshold = 0.005 # 0.5%
df['target'] = np.where(df['return'] > threshold, 1, np.where(df['return'] < -threshold, 0, np.nan))

# 데이터 준비
data_clean = df[['date'] + combined_features + ['target']].dropna().reset_index(drop=True)
print(f"Clean data shape: {data_clean.shape}")

# 학습/테스트 분할
train_mask = data_clean['date'] <= pd.to_datetime(TRAIN_END)
test_mask = data_clean['date'] > pd.to_datetime(TRAIN_END)

X_price_train = data_clean.loc[train_mask, price_features].values
X_price_test = data_clean.loc[test_mask, price_features].values
X_news_train = data_clean.loc[train_mask, news_features].values
X_news_test = data_clean.loc[test_mask, news_features].values
X_combined_train = data_clean.loc[train_mask, combined_features].values
X_combined_test = data_clean.loc[test_mask, combined_features].values

y_train = data_clean.loc[train_mask, 'target'].values
y_test = data_clean.loc[test_mask, 'target'].values

print(f"학습 샘플: {len(y_train)}, 테스트 샘플: {len(y_test)}")
print(f"가격 피쳐: {price_features}")
print(f"뉴스 피쳐: {news_features}")

# 감성분석 품질 체크
sentiment_sample = data_clean['avg_sentiment_lag_1']
print(f"\n감성 점수 분포:")
print(f"평균: {sentiment_sample.mean():.4f}")
print(f"표준편차: {sentiment_sample.std():.4f}")
print(f"긍정(>0): {(sentiment_sample > 0).sum()}/{len(sentiment_sample)}")
print(f"부정(<0): {(sentiment_sample < 0).sum()}/{len(sentiment_sample)}")
print(f"중립(=0): {(sentiment_sample == 0).sum()}/{len(sentiment_sample)}")

# 스케일링
scaler_p = StandardScaler().fit(X_price_train)
X_price_train_scaled = scaler_p.transform(X_price_train)
X_price_test_scaled = scaler_p.transform(X_price_test)

scaler_n = StandardScaler().fit(X_news_train)
X_news_train_scaled = scaler_n.transform(X_news_train)
X_news_test_scaled = scaler_n.transform(X_news_test)

scaler_c = StandardScaler().fit(X_combined_train)
X_combined_train_scaled = scaler_c.transform(X_combined_train)
X_combined_test_scaled = scaler_c.transform(X_combined_test)

# 모델 학습 및 평가
def train_models(X_train, X_test, y_train, y_test, model_prefix):
    results = []
    
    # 로지스틱 회귀
    logit = LogisticRegression(max_iter=2000, random_state=42)
    logit.fit(X_train, y_train)
    pred_logit = logit.predict(X_test)
    prob_logit = logit.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_Logit', **compute_metrics(y_test, pred_logit, prob_logit)})
    
    # 인공신경망
#     ann = MLPClassifier(hidden_layer_sizes=(20, 10), max_iter=2000, random_state=42)
#     ann.fit(X_train, y_train)
#     pred_ann = ann.predict(X_test)
#     prob_ann = ann.predict_proba(X_test)[:, 1]
#     results.append({'Model': f'{model_prefix}_ANN', **compute_metrics(y_test, pred_ann, prob_ann)})
    
    # SVM
    svm = SVC(kernel='rbf', probability=True, random_state=42)
    svm.fit(X_train, y_train)
    pred_svm = svm.predict(X_test)
    prob_svm = svm.predict_proba(X_test)[:, 1]
    results.append({'Model': f'{model_prefix}_SVM', **compute_metrics(y_test, pred_svm, prob_svm)})
    
    return results

# LSTM 모델 (시퀀스 데이터용)
def train_lstm_model(X_train, X_test, y_train, y_test, model_name, seq_len=SEQ_LEN):
    if len(X_train) < seq_len:
        return {'Model': model_name, 'Accuracy': np.nan, 'Note': 'Insufficient data for LSTM'}
    
    # 시퀀스 생성
    X_train_seq, y_train_seq = make_lstm_dataset(X_train, y_train, seq_len)
    X_test_seq, y_test_seq = make_lstm_dataset(X_test, y_test, seq_len)

    if len(X_train_seq) == 0 or len(X_test_seq) == 0:
        return {'Model': model_name, 'Accuracy': np.nan, 'Note': 'No sequences generated'}
    
    # LSTM 모델 구축 및 학습
    model = build_lstm_model(X_train_seq.shape[2])
    model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=32, verbose=0)
    
    # 예측
    prob_lstm = model.predict(X_test_seq, verbose=0)[:, 0]
    pred_lstm = (prob_lstm > 0.5).astype(int)
    
    return {'Model': model_name, **compute_metrics(y_test_seq, pred_lstm, prob_lstm)}

print("\n=== 모델 학습 및 평가 ===")

# 1. 가격 기반 모델 (P_)
price_results = train_models(X_price_train_scaled, X_price_test_scaled, y_train, y_test, 'P')

# 2. 뉴스 기반 모델 (N_)
news_results = train_models(X_news_train_scaled, X_news_test_scaled, y_train, y_test, 'N')

# 3. 결합 모델 (Combined_)
combined_results = train_models(X_combined_train_scaled, X_combined_test_scaled, y_train, y_test, 'Combined')

# 4. LSTM 모델들
lstm_price = train_lstm_model(X_price_train_scaled, X_price_test_scaled, y_train, y_test, 'P_LSTM')
lstm_news = train_lstm_model(X_news_train_scaled, X_news_test_scaled, y_train, y_test, 'N_LSTM')
lstm_combined = train_lstm_model(X_combined_train_scaled, X_combined_test_scaled, y_train, y_test, 'Combined_LSTM')

# 결과 정리
all_results = price_results + news_results + combined_results
if not pd.isna(lstm_price.get('Accuracy')):
    all_results.append(lstm_price)
if not pd.isna(lstm_news.get('Accuracy')):
    all_results.append(lstm_news)
if not pd.isna(lstm_combined.get('Accuracy')):
    all_results.append(lstm_combined)

results_df = pd.DataFrame(all_results).set_index('Model').round(4)

print("=== 최종 성능 비교 ===")
print(results_df.sort_values('Accuracy', ascending=False))

# 성능 개선 분석
price_best = results_df[results_df.index.str.startswith('P_')]['Accuracy'].max()
news_best = results_df[results_df.index.str.startswith('N_')]['Accuracy'].max()
combined_best = results_df[results_df.index.str.startswith('Combined_')]['Accuracy'].max()

print(f"\n=== 성능 개선 분석 ===")
print(f"최고 가격 모델: {price_best:.4f}")
# print(f"최고 뉴스 모델: {news_best:.4f}")
print(f"최고 결합 모델: {combined_best:.4f}")
print(f"뉴스 추가 효과: {(combined_best - price_best) * 100:.2f}%")

# 감성-수익률 상관관계 분석
sentiment_corr = np.corrcoef(data_clean['avg_sentiment_lag_1'], data_clean['target'])[0, 1]
# print(f"감성-수익률 상관계수: {sentiment_corr:.4f}")

if combined_best > price_best:
    print("✓ 뉴스 감성정보가 예측성능을 향상시켰습니다!")
else:
    print("✗ 뉴스 감성정보가 예측성능을 저하시켰습니다.")

=== 한국어 어휘기반 감성분석 (논문 방식) ===
뉴스 데이터 크기: (1698, 6)
날짜 범위: 2021-01-01 00:00:00 ~ 2025-10-02 00:00:00
평균 긍정 비율: 0.480
평균 부정 비율: 0.205
평균 중립 비율: 0.315
Clean data shape: (1440, 20)
학습 샘플: 1321, 테스트 샘플: 119
가격 피쳐: ['return_lag_1', 'return_lag_2', 'return_lag_3', 'SMA_12', 'SMA_26', 'RSI', 'MACD', 'MACD_signal', 'SMA_cross', 'MACD_hist', 'RSI_over']
뉴스 피쳐: ['pos_ratio_lag_1', 'neg_ratio_lag_1', 'avg_sentiment_lag_1']

감성 점수 분포:
평균: 0.2725
표준편차: 0.3712
긍정(>0): 1050/1440
부정(<0): 223/1440
중립(=0): 167/1440

=== 모델 학습 및 평가 ===
=== 최종 성능 비교 ===
                Accuracy  Precision  Recall      F1     AUC
Model                                                      
P_SVM             0.6303     0.6852  0.5781  0.6271  0.6381
Combined_SVM      0.6050     0.7073  0.4531  0.5524  0.6116
P_Logit           0.5966     0.6481  0.5469  0.5932  0.6750
P_LSTM            0.5862     0.6296  0.5484  0.5862  0.6395
Combined_Logit    0.5798     0.6296  0.5312  0.5763  0.6815
N_LSTM            0.5690     0.5789  0.70